## Install Spark & Install Packages & Initial Spark

In [ ]:
!pip install pyspark==3.0.1 py4j==0.10.9 

In [ ]:
import numpy as np
import pandas as pd 
import warnings
import zipfile
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties
from pyspark.sql import functions as F
from pyspark.sql import Window

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Feature Engineering(Counts & Ratio)")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

## Read Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
train = spark.read.option("header",True) \
    .csv("/content/drive/MyDrive/Colab Notebooks/data/data_format1/train_format1.csv")
test = spark.read.option("header",True) \
    .csv("/content/drive/MyDrive/Colab Notebooks/data/data_format1/test_format1.csv")
train_info = spark.read.option("header",True) \
    .csv("/content/drive/MyDrive/Colab Notebooks/data/data_format1/user_info_format1.csv") 
train_log = spark.read.option("header",True) \
    .csv("/content/drive/MyDrive/Colab Notebooks/data/data_format1/user_log_format1.csv") \
    .withColumnRenamed("seller_id", "merchant_id")

In [ ]:
test.count()

261477

In [ ]:
train_info_0 = train_info \
    .withColumn("age", F.when(train_info.age_range == 8, 7).otherwise(train_info.age_range)) \
    .fillna({"age": 0, "gender": 2}) \
    .drop("age_range")

In [ ]:
train_log_0 = train_log \
            .withColumn("action_type_tmp", F.when(train_log.action_type == 0, 1).otherwise(train_log.action_type)) \
            .drop("action_type") \
            .withColumnRenamed("action_type_tmp", "action_type") \
            .fillna({"brand_id": -1})

In [ ]:
user_feature = spark.read.option("header",True) \
    .parquet("/content/drive/MyDrive/Colab Notebooks/data/feature_user_new")
mer_feature = spark.read.option("header",True) \
    .parquet("/content/drive/MyDrive/Colab Notebooks/data/feature_mer_new")
brd_feature = spark.read.option("header",True) \
    .parquet("/content/drive/MyDrive/Colab Notebooks/data/feature_brand_new")
cat_feature = spark.read.option("header",True) \
    .parquet("/content/drive/MyDrive/Colab Notebooks/data/feature_cat_new")
itm_feature = spark.read.option("header",True) \
    .parquet("/content/drive/MyDrive/Colab Notebooks/data/feature_item")
um_feature = spark.read.option("header",True) \
    .parquet("/content/drive/MyDrive/Colab Notebooks/data/feature_um_new")
mb_feature = spark.read.option("header",True) \
    .parquet("/content/drive/MyDrive/Colab Notebooks/data/feature_mb")
mc_feature = spark.read.option("header",True) \
    .parquet("/content/drive/MyDrive/Colab Notebooks/data/feature_mc")

## Show Data

In [ ]:
train.orderBy("user_id", "merchant_id").show(10)

+-------+-----------+-----+
|user_id|merchant_id|label|
+-------+-----------+-----+
|      1|       1019|    1|
|   1000|       3819|    0|
|   1000|        598|    0|
| 100001|       1963|    0|
| 100001|       2273|    0|
| 100002|        915|    0|
| 100005|       1618|    0|
| 100006|       4847|    0|
| 100009|        386|    0|
|  10001|       1760|    0|
+-------+-----------+-----+
only showing top 10 rows



In [ ]:
train_info_0.show(1)

+-------+------+---+
|user_id|gender|age|
+-------+------+---+
| 376517|     1|  6|
+-------+------+---+
only showing top 1 row



In [ ]:
train_log.orderBy("user_id", "merchant_id").show(10)

+-------+-------+------+-----------+--------+----------+-----------+
|user_id|item_id|cat_id|merchant_id|brand_id|time_stamp|action_type|
+-------+-------+------+-----------+--------+----------+-----------+
|      1|1110495|   992|       1019|    6805|      1111|          0|
|      1|1110495|   992|       1019|    6805|      1111|          0|
|      1|1110495|   992|       1019|    6805|      1111|          0|
|      1|1110495|   992|       1019|    6805|      1111|          2|
|      1|1110495|   992|       1019|    6805|      1111|          2|
|      1|1110495|   992|       1019|    6805|      1111|          2|
|      1|1110495|   992|       1019|    6805|      1111|          0|
|      1|1110495|   992|       1019|    6805|      1111|          2|
|      1|1110495|   992|       1019|    6805|      1111|          0|
|      1|1110495|   992|       1019|    6805|      1111|          0|
+-------+-------+------+-----------+--------+----------+-----------+
only showing top 10 rows



In [ ]:
user_feature.orderBy("user_id").show(10)

+-------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+---------------

In [ ]:
mer_feature.orderBy("merchant_id").show(10)

+-----------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+-----------

In [ ]:
brd_feature.orderBy("brand_id").show(10)

+--------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+--------------

In [ ]:
itm_feature.show()

+-------+-----------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+-------------------+---------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-----------------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+----------------+----------------+--------

In [ ]:
um_feature.orderBy("user_id", "merchant_id").show(10)

+-------+-----------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+------------

## Gathering
Expanded the training/testing data by adding age range and gender of users, item id, brand id, and category id, where item id is the id of the item bought by the user from the merchant on the Double 11 day.

In [ ]:
train_log_selected_0 = train_log \
                    .filter((train_log.time_stamp == "1111") & (train_log.action_type == 2)) \
                    .groupBy("user_id", "merchant_id", "item_id", "cat_id", "brand_id") \
                    .agg(F.count(F.lit(1)).alias("item_freq"))

train_log_selected_1 = train_log_selected_0 \
                       .groupBy("user_id", "merchant_id") \
                       .agg(F.max("item_freq").alias("item_freq_max"))

train_log_selected_2 = train_log_selected_0 \
                       .join(train_log_selected_1, ["user_id", "merchant_id"], "left") \
                       .filter(F.col("item_freq") == F.col("item_freq_max")) \
                       .groupBy("user_id", "merchant_id") \
                       .agg(F.first("item_id").alias("item_id"), F.first("cat_id").alias("cat_id"), F.first("brand_id").alias("brand_id"))

train_log_selected_3 = train_log_selected_2 \
                       .join(train_log.drop(*["time_stamp", "action_type"]), ["user_id", "merchant_id", "item_id", "cat_id", "brand_id"], "left") \
                       .distinct()     

In [ ]:
test_feature_0 = test \
                  .join(train_info_0, "user_id", "left") \
                  .join(train_log_selected_3, ["user_id", "merchant_id"], "left") \
                  .orderBy("user_id", "merchant_id")

# +-------+-----------+-----+------+---+-------+------+--------+
# |user_id|merchant_id|label|gender|age|item_id|cat_id|brand_id|
# +-------+-----------+-----+------+---+-------+------+--------+
# |      1|       1019|    1|     1|  3|1110495|   992|    6805|
# |   1000|       3819|    0|     1|  2| 517962|     2|    8055|
# |   1000|        598|    0|     1|  2| 708788|   300|    6983|
# | 100001|       1963|    0|     1|  0|1070887|  1023|    6109|
# | 100001|       2273|    0|     1|  0| 191499|  1421|    2240|
# | 100002|        915|    0|     0|  4| 373571|  1122|    8062|
# | 100005|       1618|    0|     0|  3| 331371|  1611|    8235|
# | 100006|       4847|    0|     0|  4| 779344|  1577|    1988|
# | 100009|        386|    0|     1|  3| 137180|   592|    3302|
# |  10001|       1760|    0|     0|  2| 528905|   602|    2197|
# | 100011|       1923|    0|     0|  4| 497208|   656|    4184|
# | 100011|       2700|    0|     0|  4| 625048|  1280|    5476|
# | 100012|       3160|    0|     0|  2|   4648|  1505|    4065|
# | 100014|       1073|    1|     0|  6| 780896|   766|    6748|
# | 100015|       4282|    0|     1|  3| 687044|  1438|    7892|
# | 100016|       1710|    1|     0|  4| 772774|   898|    2946|
# | 100017|       1089|    0|     0|  0| 225794|   656|    2850|
# | 100019|        643|    0|     1|  3| 573770|  1553|     968|
# |  10002|       4744|    0|     0|  3| 185161|   178|    5022|
# | 100022|       2303|    0|     0|  2| 966737|   662|    7011|
# +-------+-----------+-----+------+---+-------+------+--------+


## Gathering

In [ ]:
test_feature_1 = test_feature_0 \
                .join(user_feature, "user_id", "left") \
                .orderBy("user_id", "merchant_id") 
# 260864
# +-------+-----------+-----+------+---+-------+------+--------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+
# |user_id|merchant_id|label|gender|age|item_id|cat_id|brand_id|u_ttl_clk_cnt|u_ttl_atc_cnt|u_ttl_buy_cnt|u_ttl_atf_cnt|u_clk_cnt_m5|u_clk_cnt_m6|u_clk_cnt_m7|u_clk_cnt_m8|u_clk_cnt_m9|u_clk_cnt_m10|u_clk_cnt_m11|u_atc_cnt_m5|u_atc_cnt_m6|u_atc_cnt_m7|u_atc_cnt_m8|u_atc_cnt_m9|u_atc_cnt_m10|u_atc_cnt_m11|u_buy_cnt_m5|u_buy_cnt_m6|u_buy_cnt_m7|u_buy_cnt_m8|u_buy_cnt_m9|u_buy_cnt_m10|u_buy_cnt_m11|u_atf_cnt_m5|u_atf_cnt_m6|u_atf_cnt_m7|u_atf_cnt_m8|u_atf_cnt_m9|u_atf_cnt_m10|u_atf_cnt_m11|u_ttl_clk_ratio|u_ttl_atc_ratio|u_ttl_buy_ratio|u_ttl_atf_ratio|u_clk_ratio_m5|u_clk_ratio_m6|u_clk_ratio_m7|u_clk_ratio_m8|u_clk_ratio_m9|u_clk_ratio_m10|u_clk_ratio_m11|u_atc_ratio_m5|u_atc_ratio_m6|u_atc_ratio_m7|u_atc_ratio_m8|u_atc_ratio_m9|u_atc_ratio_m10|u_atc_ratio_m11|u_buy_ratio_m5|u_buy_ratio_m6|u_buy_ratio_m7|u_buy_ratio_m8|u_buy_ratio_m9|u_buy_ratio_m10|u_buy_ratio_m11|u_atf_ratio_m5|u_atf_ratio_m6|u_atf_ratio_m7|u_atf_ratio_m8|u_atf_ratio_m9|u_atf_ratio_m10|u_atf_ratio_m11|u_clk_day_cnt|u_atc_day_cnt|u_buy_day_cnt|u_atf_day_cnt|u_clk_day_cnt_m5|u_clk_day_cnt_m6|u_clk_day_cnt_m7|u_clk_day_cnt_m8|u_clk_day_cnt_m9|u_clk_day_cnt_m10|u_clk_day_cnt_m11|u_atc_day_cnt_m5|u_atc_day_cnt_m6|u_atc_day_cnt_m7|u_atc_day_cnt_m8|u_atc_day_cnt_m9|u_atc_day_cnt_m10|u_atc_day_cnt_m11|u_buy_day_cnt_m5|u_buy_day_cnt_m6|u_buy_day_cnt_m7|u_buy_day_cnt_m8|u_buy_day_cnt_m9|u_buy_day_cnt_m10|u_buy_day_cnt_m11|u_atf_day_cnt_m5|u_atf_day_cnt_m6|u_atf_day_cnt_m7|u_atf_day_cnt_m8|u_atf_day_cnt_m9|u_atf_day_cnt_m10|u_atf_day_cnt_m11|u_clk_item_div|u_atc_item_div|u_buy_item_div|u_atf_item_div|u_clk_cat_div|u_atc_cat_div|u_buy_cat_div|u_atf_cat_div|u_clk_brd_div|u_atc_brd_div|u_buy_brd_div|u_atf_brd_div|u_clk_mer_div|u_atc_mer_div|u_buy_mer_div|u_atf_mer_div|u_clk_mean|u_atc_mean|u_buy_mean|u_atf_mean|u_clk_median|u_atc_median|u_buy_median|u_atf_median|u_clk_std|u_atc_std|u_buy_std|u_atf_std|u_clk_max|u_atc_max|u_buy_max|u_atf_max|u_clk_day_cnt_mean|u_clk_day_cnt_max|u_clk_day_cnt_std|u_clk_day_cnt_med|u_atc_day_cnt_mean|u_atc_day_cnt_max|u_atc_day_cnt_std|u_atc_day_cnt_med|u_buy_day_cnt_mean|u_buy_day_cnt_max|u_buy_day_cnt_std|u_buy_day_cnt_med|u_atf_day_cnt_mean|u_atf_day_cnt_max|u_atf_day_cnt_std|u_atf_day_cnt_med|u_clk_item_cnt_mean|u_clk_item_cnt_max|u_clk_item_cnt_std|u_clk_item_cnt_med|u_atc_item_cnt_mean|u_atc_item_cnt_max|u_atc_item_cnt_std|u_atc_item_cnt_med|u_buy_item_cnt_mean|u_buy_item_cnt_max|u_buy_item_cnt_std|u_buy_item_cnt_med|u_atf_item_cnt_mean|u_atf_item_cnt_max|u_atf_item_cnt_std|u_atf_item_cnt_med|u_clk_cnt_double11|u_atc_cnt_double11|u_buy_cnt_double11|u_atf_cnt_double11|u_clk_cnt_1wpre_double11|u_atc_cnt_1wpre_double11|u_buy_cnt_1wpre_double11|u_atf_cnt_1wpre_double11|u_clk_ratio_double11|u_clk_ratio_1wpre_double11|u_atc_ratio_double11|u_atc_ratio_1wpre_double11|u_buy_ratio_double11|u_buy_ratio_1wpre_double11|u_atf_ratio_double11|u_atf_ratio_1wpre_double11|
# +-------+-----------+-----+------+---+-------+------+--------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+
# |      1|       1019|    1|     1|  3|1110495|   992|    6805|           27|            0|            6|            0|           0|           0|           0|           0|           0|           14|           13|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            2|            4|           0|           0|           0|           0|           0|            0|            0|           0.82|            0.0|           0.18|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|           0.88|           0.76|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|           0.13|           0.24|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            5|            0|            3|            0|               0|               0|               0|               0|               0|                4|                1|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                2|                1|               0|               0|               0|               0|               0|                0|                0|            12|             0|             3|             0|            6|            0|            3|            0|            9|            0|            3|            0|            9|            0|            3|            0|      2.33|       0.0|      0.33|       0.0|         0.0|         0.0|         0.0|         0.0|     5.72|      0.0|     0.82|      0.0|       14|        0|        2|        0|              1.11|                2|             0.33|              1.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.33|                 4|               1.0|               1.0|                0.0|                 0|               0.0|               0.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                13|                 0|                 4|                 0|                       0|                       0|                       0|                       0|                0.76|                       0.0|                 0.0|                       0.0|                0.24|                       0.0|                 0.0|                       0.0|
# |   1000|       3819|    0|     1|  2| 517962|     2|    8055|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|
# |   1000|        598|    0|     1|  2| 708788|   300|    6983|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|
# |   1000|        598|    0|     1|  2| 708788|   300|    8351|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|
# | 100001|       1963|    0|     1|  0|1070887|  1023|    6109|           87|            0|            3|            0|           0|           0|           0|           1|           1|            1|           84|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            3|           0|           0|           0|           0|           0|            0|            0|           0.97|            0.0|           0.03|            0.0|           0.0|           0.0|           0.0|           1.0|           1.0|            1.0|           0.97|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.03|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            4|            0|            1|            0|               0|               0|               0|               1|               1|                1|                1|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                0|            74|             0|             3|             0|           20|            0|            3|            0|           35|            0|            3|            0|           40|            0|            3|            0|       0.5|       0.0|       0.0|       0.0|         0.5|         0.0|         0.0|         0.0|     0.55|      0.0|      0.0|      0.0|        1|        0|        0|        0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.85|                10|              1.97|               1.0|                0.0|                 0|               0.0|               0.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                84|                 0|                 3|                 0|                       0|                       0|                       0|                       0|                0.97|                       0.0|                 0.0|                       0.0|                0.03|                       0.0|                 0.0|                       0.0|
# +-------+-----------+-----+------+---+-------+------+--------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+


In [ ]:
test_feature_2 = test_feature_1 \
                .join(mer_feature, "merchant_id", "left") \
                .orderBy("merchant_id")

# +-----------+-------+-----+------+---+-------+------+--------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+----------------+----------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+
# |merchant_id|user_id|label|gender|age|item_id|cat_id|brand_id|u_ttl_clk_cnt|u_ttl_atc_cnt|u_ttl_buy_cnt|u_ttl_atf_cnt|u_clk_cnt_m5|u_clk_cnt_m6|u_clk_cnt_m7|u_clk_cnt_m8|u_clk_cnt_m9|u_clk_cnt_m10|u_clk_cnt_m11|u_atc_cnt_m5|u_atc_cnt_m6|u_atc_cnt_m7|u_atc_cnt_m8|u_atc_cnt_m9|u_atc_cnt_m10|u_atc_cnt_m11|u_buy_cnt_m5|u_buy_cnt_m6|u_buy_cnt_m7|u_buy_cnt_m8|u_buy_cnt_m9|u_buy_cnt_m10|u_buy_cnt_m11|u_atf_cnt_m5|u_atf_cnt_m6|u_atf_cnt_m7|u_atf_cnt_m8|u_atf_cnt_m9|u_atf_cnt_m10|u_atf_cnt_m11|u_ttl_clk_ratio|u_ttl_atc_ratio|u_ttl_buy_ratio|u_ttl_atf_ratio|u_clk_ratio_m5|u_clk_ratio_m6|u_clk_ratio_m7|u_clk_ratio_m8|u_clk_ratio_m9|u_clk_ratio_m10|u_clk_ratio_m11|u_atc_ratio_m5|u_atc_ratio_m6|u_atc_ratio_m7|u_atc_ratio_m8|u_atc_ratio_m9|u_atc_ratio_m10|u_atc_ratio_m11|u_buy_ratio_m5|u_buy_ratio_m6|u_buy_ratio_m7|u_buy_ratio_m8|u_buy_ratio_m9|u_buy_ratio_m10|u_buy_ratio_m11|u_atf_ratio_m5|u_atf_ratio_m6|u_atf_ratio_m7|u_atf_ratio_m8|u_atf_ratio_m9|u_atf_ratio_m10|u_atf_ratio_m11|u_clk_day_cnt|u_atc_day_cnt|u_buy_day_cnt|u_atf_day_cnt|u_clk_day_cnt_m5|u_clk_day_cnt_m6|u_clk_day_cnt_m7|u_clk_day_cnt_m8|u_clk_day_cnt_m9|u_clk_day_cnt_m10|u_clk_day_cnt_m11|u_atc_day_cnt_m5|u_atc_day_cnt_m6|u_atc_day_cnt_m7|u_atc_day_cnt_m8|u_atc_day_cnt_m9|u_atc_day_cnt_m10|u_atc_day_cnt_m11|u_buy_day_cnt_m5|u_buy_day_cnt_m6|u_buy_day_cnt_m7|u_buy_day_cnt_m8|u_buy_day_cnt_m9|u_buy_day_cnt_m10|u_buy_day_cnt_m11|u_atf_day_cnt_m5|u_atf_day_cnt_m6|u_atf_day_cnt_m7|u_atf_day_cnt_m8|u_atf_day_cnt_m9|u_atf_day_cnt_m10|u_atf_day_cnt_m11|u_clk_item_div|u_atc_item_div|u_buy_item_div|u_atf_item_div|u_clk_cat_div|u_atc_cat_div|u_buy_cat_div|u_atf_cat_div|u_clk_brd_div|u_atc_brd_div|u_buy_brd_div|u_atf_brd_div|u_clk_mer_div|u_atc_mer_div|u_buy_mer_div|u_atf_mer_div|u_clk_mean|u_atc_mean|u_buy_mean|u_atf_mean|u_clk_median|u_atc_median|u_buy_median|u_atf_median|u_clk_std|u_atc_std|u_buy_std|u_atf_std|u_clk_max|u_atc_max|u_buy_max|u_atf_max|u_clk_day_cnt_mean|u_clk_day_cnt_max|u_clk_day_cnt_std|u_clk_day_cnt_med|u_atc_day_cnt_mean|u_atc_day_cnt_max|u_atc_day_cnt_std|u_atc_day_cnt_med|u_buy_day_cnt_mean|u_buy_day_cnt_max|u_buy_day_cnt_std|u_buy_day_cnt_med|u_atf_day_cnt_mean|u_atf_day_cnt_max|u_atf_day_cnt_std|u_atf_day_cnt_med|u_clk_item_cnt_mean|u_clk_item_cnt_max|u_clk_item_cnt_std|u_clk_item_cnt_med|u_atc_item_cnt_mean|u_atc_item_cnt_max|u_atc_item_cnt_std|u_atc_item_cnt_med|u_buy_item_cnt_mean|u_buy_item_cnt_max|u_buy_item_cnt_std|u_buy_item_cnt_med|u_atf_item_cnt_mean|u_atf_item_cnt_max|u_atf_item_cnt_std|u_atf_item_cnt_med|u_clk_cnt_double11|u_atc_cnt_double11|u_buy_cnt_double11|u_atf_cnt_double11|u_clk_cnt_1wpre_double11|u_atc_cnt_1wpre_double11|u_buy_cnt_1wpre_double11|u_atf_cnt_1wpre_double11|u_clk_ratio_double11|u_clk_ratio_1wpre_double11|u_atc_ratio_double11|u_atc_ratio_1wpre_double11|u_buy_ratio_double11|u_buy_ratio_1wpre_double11|u_atf_ratio_double11|u_atf_ratio_1wpre_double11|m_ttl_clk_cnt|m_ttl_atc_cnt|m_ttl_buy_cnt|m_ttl_atf_cnt|m_clk_cnt_m5|m_clk_cnt_m6|m_clk_cnt_m7|m_clk_cnt_m8|m_clk_cnt_m9|m_clk_cnt_m10|m_clk_cnt_m11|m_atc_cnt_m5|m_atc_cnt_m6|m_atc_cnt_m7|m_atc_cnt_m8|m_atc_cnt_m9|m_atc_cnt_m10|m_atc_cnt_m11|m_buy_cnt_m5|m_buy_cnt_m6|m_buy_cnt_m7|m_buy_cnt_m8|m_buy_cnt_m9|m_buy_cnt_m10|m_buy_cnt_m11|m_atf_cnt_m5|m_atf_cnt_m6|m_atf_cnt_m7|m_atf_cnt_m8|m_atf_cnt_m9|m_atf_cnt_m10|m_atf_cnt_m11|m_ttl_clk_ratio|m_ttl_atc_ratio|m_ttl_buy_ratio|m_ttl_atf_ratio|m_clk_ratio_m5|m_clk_ratio_m6|m_clk_ratio_m7|m_clk_ratio_m8|m_clk_ratio_m9|m_clk_ratio_m10|m_clk_ratio_m11|m_atc_ratio_m5|m_atc_ratio_m6|m_atc_ratio_m7|m_atc_ratio_m8|m_atc_ratio_m9|m_atc_ratio_m10|m_atc_ratio_m11|m_buy_ratio_m5|m_buy_ratio_m6|m_buy_ratio_m7|m_buy_ratio_m8|m_buy_ratio_m9|m_buy_ratio_m10|m_buy_ratio_m11|m_atf_ratio_m5|m_atf_ratio_m6|m_atf_ratio_m7|m_atf_ratio_m8|m_atf_ratio_m9|m_atf_ratio_m10|m_atf_ratio_m11|m_clk_day_cnt|m_atc_day_cnt|m_buy_day_cnt|m_atf_day_cnt|m_clk_day_cnt_m5|m_clk_day_cnt_m6|m_clk_day_cnt_m7|m_clk_day_cnt_m8|m_clk_day_cnt_m9|m_clk_day_cnt_m10|m_clk_day_cnt_m11|m_atc_day_cnt_m5|m_atc_day_cnt_m6|m_atc_day_cnt_m7|m_atc_day_cnt_m8|m_atc_day_cnt_m9|m_atc_day_cnt_m10|m_atc_day_cnt_m11|m_buy_day_cnt_m5|m_buy_day_cnt_m6|m_buy_day_cnt_m7|m_buy_day_cnt_m8|m_buy_day_cnt_m9|m_buy_day_cnt_m10|m_buy_day_cnt_m11|m_atf_day_cnt_m5|m_atf_day_cnt_m6|m_atf_day_cnt_m7|m_atf_day_cnt_m8|m_atf_day_cnt_m9|m_atf_day_cnt_m10|m_atf_day_cnt_m11|m_clk_item_div|m_atc_item_div|m_buy_item_div|m_atf_item_div|m_clk_cat_div|m_atc_cat_div|m_buy_cat_div|m_atf_cat_div|m_clk_brd_div|m_atc_brd_div|m_buy_brd_div|m_atf_brd_div|m_clk_mer_div|m_atc_mer_div|m_buy_mer_div|m_atf_mer_div|m_clk_mean|m_atc_mean|m_buy_mean|m_atf_mean|m_clk_median|m_atc_median|m_buy_median|m_atf_median|m_clk_std|m_atc_std|m_buy_std|m_atf_std|m_clk_max|m_atc_max|m_buy_max|m_atf_max|m_clk_day_cnt_mean|m_clk_day_cnt_max|m_clk_day_cnt_std|m_clk_day_cnt_med|m_atc_day_cnt_mean|m_atc_day_cnt_max|m_atc_day_cnt_std|m_atc_day_cnt_med|m_buy_day_cnt_mean|m_buy_day_cnt_max|m_buy_day_cnt_std|m_buy_day_cnt_med|m_atf_day_cnt_mean|m_atf_day_cnt_max|m_atf_day_cnt_std|m_atf_day_cnt_med|m_clk_item_cnt_mean|m_clk_item_cnt_max|m_clk_item_cnt_std|m_clk_item_cnt_med|m_atc_item_cnt_mean|m_atc_item_cnt_max|m_atc_item_cnt_std|m_atc_item_cnt_med|m_buy_item_cnt_mean|m_buy_item_cnt_max|m_buy_item_cnt_std|m_buy_item_cnt_med|m_atf_item_cnt_mean|m_atf_item_cnt_max|m_atf_item_cnt_std|m_atf_item_cnt_med|m_clk_cnt_double11|m_atc_cnt_double11|m_buy_cnt_double11|m_atf_cnt_double11|m_clk_cnt_1wpre_double11|m_atc_cnt_1wpre_double11|m_buy_cnt_1wpre_double11|m_atf_cnt_1wpre_double11|ttl_cnt_double11|ttl_cnt_1wpre_double11|m_clk_ratio_double11|m_clk_ratio_1wpre_double11|m_atc_ratio_double11|m_atc_ratio_1wpre_double11|m_buy_ratio_double11|m_buy_ratio_1wpre_double11|m_atf_ratio_double11|m_atf_ratio_1wpre_double11|m_rpt_byr_num|m_rpt_byr_day|m_popular_age|m_popular_gender|
# +-----------+-------+-----+------+---+-------+------+--------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+----------------+----------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+
# |       1019|      1|    1|     1|  3|1110495|   992|    6805|           27|            0|            6|            0|           0|           0|           0|           0|           0|           14|           13|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            2|            4|           0|           0|           0|           0|           0|            0|            0|           0.82|            0.0|           0.18|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|           0.88|           0.76|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|           0.13|           0.24|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            5|            0|            3|            0|               0|               0|               0|               0|               0|                4|                1|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                2|                1|               0|               0|               0|               0|               0|                0|                0|            12|             0|             3|             0|            6|            0|            3|            0|            9|            0|            3|            0|            9|            0|            3|            0|      2.33|       0.0|      0.33|       0.0|         0.0|         0.0|         0.0|         0.0|     5.72|      0.0|     0.82|      0.0|       14|        0|        2|        0|              1.11|                2|             0.33|              1.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.33|                 4|               1.0|               1.0|                0.0|                 0|               0.0|               0.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                13|                 0|                 4|                 0|                       0|                       0|                       0|                       0|                0.76|                       0.0|                 0.0|                       0.0|                0.24|                       0.0|                 0.0|                       0.0|         6003|            5|          941|          484|         463|         678|         459|         511|         727|         1263|         1902|           0|           0|           1|           1|           0|            2|            1|         182|          83|          38|          67|          96|          180|          295|          85|          53|          32|          35|          48|          127|          104|           0.81|            0.0|           0.13|           0.07|          0.63|          0.83|          0.87|          0.83|          0.83|            0.8|           0.83|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.25|           0.1|          0.07|          0.11|          0.11|           0.11|           0.13|          0.12|          0.07|          0.06|          0.06|          0.06|           0.08|           0.05|          174|            5|          139|          142|              12|              30|              29|              31|              30|               31|               11|               0|               0|               1|               1|               0|                2|                1|              20|              20|              15|              20|              24|               29|               11|              21|              23|              17|              20|              20|               31|               10|           147|             3|            61|            60|            8|            2|            8|            8|            1|            0|            1|            1|            1|            1|            1|            1|     683.5|      0.67|    107.67|     63.33|       594.5|         0.5|        89.5|        50.5|   305.46|     0.82|    60.02|    36.47|     1263|        2|      182|      127|              1.17|               10|             0.57|              1.0|               1.0|                1|              0.0|              1.0|              1.05|                3|             0.23|              1.0|              1.04|                3|             0.21|              1.0|               1.33|                14|              0.99|               1.0|                1.0|                 1|               0.0|               1.0|               1.16|                 4|              0.45|               1.0|               1.07|                 6|              0.37|               1.0|              1167|                 1|               234|                37|                     544|                       0|                      40|                      51|            1439|                   635|                0.81|                      0.86|                 0.0|                       0.0|                0.16|                      0.06|                0.03|                      0.08|          191|          215|            3|               0|
# |       3819|   1000|    0|     1|  2| 517962|     2|    8055|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|         6642|           23|          467|          335|          15|          86|         102|         253|         384|          813|         4989|           0|           0|           0|           0|           1|            1|           21|           0|           2|           3|           9|          11|           21|          421|           4|           7|           7|          17|          26|           73|          201|           0.89|            0.0|           0.06|           0.04|          0.79|          0.91|          0.91|          0.91|          0.91|            0.9|           0.89|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|          0.02|          0.03|          0.03|          0.03|           0.02|           0.07|          0.21|          0.07|          0.06|          0.06|          0.06|           0.08|           0.04|          153|            8|           37|           75|               7|              22|              22|              30|              30|               31|               11|               0|               0|               0|               0|               1|                1|                6|               0|               2|               2|               5|               8|               12|                8|               3|               5|               6|              12|              14|               24|               11|            95|            12|            40|            38|           14|            6|           12|           11|            1|            0|            1|            1|            1|            1|            1|            1|     275.5|      0.33|      7.67|     22.33|       177.5|         0.0|         6.0|        12.0|   294.98|     0.52|     7.79|    26.14|      813|        1|       21|       73|              1.26|               13|             0.76|              1.0|              1.05|                2|             0.22|              1.0|              1.02|                3|             0.16|              1.0|               1.1|                3|             0.32|              1.0|               1.88|                29|              2.09|               1.0|                1.1|                 2|              0.31|               1.0|               1.21|                 4|              0.53|               1.0|               1.25|                 5|              0.63|               1.0|              2575|                 7|               408|                41|                    2160|                      12|                      11|                     142|            3031|                  2325|                0.85|                      0.93|                 0.0|                      0.01|                0.13|                       0.0|                0.01|                      0.06|           65|           73|            3|               1|
# |        598|   1000|    0|     1|  2| 708788|   300|    6983|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|        82733|          206|         4727|         4229|        3650|        9470|        4960|        4571|        7028|        13390|        39664|           8|           4|           3|           4|           3|           18|          166|         174|         285|         178|         118|         258|          301|         3413|         333|         452|         280|         206|         351|          958|         1649|            0.9|            0.0|           0.05|           0.05|          0.88|          0.93|          0.91|          0.93|          0.92|           0.91|           0.88|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.04|          0.03|          0.03|          0.02|          0.03|           0.02|           0.08|          0.08|          0.04|          0.05|          0.04|          0.05|           0.07|           0.04|          176|           42|          174|          185|              12|              30|              31|              31|              30|               31|               11|               6|               3|               3|               4|               3|               13|               10|              21|              29|              30|              25|              29|               31|                9|              21|              30|              31|              31|              30|               31|               11|           728|            92|           370|           462|           12|            5|            8|           10|            1|            0|            1|            1|            1|            1|            1|            1|   7178.17|      6.67|     219.0|     430.0|      5994.0|         4.0|       218.0|       342.0|  3688.56|     5.85|    72.81|   271.14|    13390|       18|      301|      958|              1.69|               48|             1.69|              1.0|              1.08|                3|             0.29|              1.0|              1.05|                6|             0.26|              1.0|              1.27|               12|             0.73|              1.0|                3.1|               124|              4.77|               1.0|               1.27|                 4|              0.64|               1.0|               1.13|                 8|              0.46|               1.0|               1.61|                25|              1.48|               1.0|             20505|                58|              3393|               310|                   16070|                      86|                      13|                    1064|           24266|                 17233|                0.85|                      0.93|                 0.0|                       0.0|                0.14|                       0.0|                0.01|                      0.06|          528|          721|            3|               1|
# |        598|   1000|    0|     1|  2| 708788|   300|    8351|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|        82733|          206|         4727|         4229|        3650|        9470|        4960|        4571|        7028|        13390|        39664|           8|           4|           3|           4|           3|           18|          166|         174|         285|         178|         118|         258|          301|         3413|         333|         452|         280|         206|         351|          958|         1649|            0.9|            0.0|           0.05|           0.05|          0.88|          0.93|          0.91|          0.93|          0.92|           0.91|           0.88|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.04|          0.03|          0.03|          0.02|          0.03|           0.02|           0.08|          0.08|          0.04|          0.05|          0.04|          0.05|           0.07|           0.04|          176|           42|          174|          185|              12|              30|              31|              31|              30|               31|               11|               6|               3|               3|               4|               3|               13|               10|              21|              29|              30|              25|              29|               31|                9|              21|              30|              31|              31|              30|               31|               11|           728|            92|           370|           462|           12|            5|            8|           10|            1|            0|            1|            1|            1|            1|            1|            1|   7178.17|      6.67|     219.0|     430.0|      5994.0|         4.0|       218.0|       342.0|  3688.56|     5.85|    72.81|   271.14|    13390|       18|      301|      958|              1.69|               48|             1.69|              1.0|              1.08|                3|             0.29|              1.0|              1.05|                6|             0.26|              1.0|              1.27|               12|             0.73|              1.0|                3.1|               124|              4.77|               1.0|               1.27|                 4|              0.64|               1.0|               1.13|                 8|              0.46|               1.0|               1.61|                25|              1.48|               1.0|             20505|                58|              3393|               310|                   16070|                      86|                      13|                    1064|           24266|                 17233|                0.85|                      0.93|                 0.0|                       0.0|                0.14|                       0.0|                0.01|                      0.06|          528|          721|            3|               1|
# |       1963| 100001|    0|     1|  0|1070887|  1023|    6109|           87|            0|            3|            0|           0|           0|           0|           1|           1|            1|           84|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            3|           0|           0|           0|           0|           0|            0|            0|           0.97|            0.0|           0.03|            0.0|           0.0|           0.0|           0.0|           1.0|           1.0|            1.0|           0.97|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.03|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            4|            0|            1|            0|               0|               0|               0|               1|               1|                1|                1|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                0|            74|             0|             3|             0|           20|            0|            3|            0|           35|            0|            3|            0|           40|            0|            3|            0|       0.5|       0.0|       0.0|       0.0|         0.5|         0.0|         0.0|         0.0|     0.55|      0.0|      0.0|      0.0|        1|        0|        0|        0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.85|                10|              1.97|               1.0|                0.0|                 0|               0.0|               0.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                84|                 0|                 3|                 0|                       0|                       0|                       0|                       0|                0.97|                       0.0|                 0.0|                       0.0|                0.03|                       0.0|                 0.0|                       0.0|        57321|           54|         1497|         1553|        2393|        5929|        1471|        1858|        2206|         5332|        38132|           0|           3|           1|           1|           1|            5|           43|          89|          81|          10|          16|          18|           13|         1270|         126|         204|          59|          51|          64|          309|          740|           0.95|            0.0|           0.02|           0.03|          0.92|          0.95|          0.95|          0.96|          0.96|           0.94|           0.95|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.03|          0.01|          0.01|          0.01|          0.01|            0.0|           0.03|          0.05|          0.03|          0.04|          0.03|          0.03|           0.05|           0.02|          176|           22|           77|          153|              12|              30|              31|              31|              30|               31|               11|               0|               3|               1|               1|               1|                5|               11|              18|              18|               8|               8|              13|               11|                1|              20|              28|              19|              21|              25|               29|               11|           500|            41|           190|           324|           28|            7|           17|           24|            1|            0|            1|            1|            1|            1|            1|            1|   3198.17|      1.83|     37.83|     135.5|      2299.5|         1.0|        17.0|        95.0|  1919.51|     1.83|    36.72|   102.95|     5929|        5|       89|      309|              1.54|               35|             1.42|              1.0|              1.11|                2|             0.32|              1.0|              1.02|                3|             0.13|              1.0|              1.27|               10|             0.75|              1.0|               3.75|               119|              6.05|               2.0|               1.23|                 4|              0.64|               1.0|               1.21|                 7|              0.57|               1.0|               1.81|                22|              1.75|               1.0|             20983|                 8|              1270|               102|                   13743|                      25|                       0|                     494|           22363|                 14262|                0.94|                      0.96|                 0.0|                       0.0|                0.06|                       0.0|                 0.0|                      0.03|          205|          224|            3|               1|
# +-----------+-------+-----+------+---+-------+------+--------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+----------------+----------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+


In [ ]:
test_feature_3 = test_feature_2 \
                .join(um_feature, ["user_id", "merchant_id"], "left") \
                .orderBy("user_id", "merchant_id")

# +-------+-----------+-----+------+---+-------+------+--------+-----------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+-------------------+---------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-----------------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------------+--------------+--------------+-----------------+-------------+-------------+-----------------+-------------+-------------+-----------------+-------------+-------------+--------------+----------+----------+----------------+------------+------------+-------------+---------+---------+-------------+---------+---------+----------------------+---------------------+---------------------+---------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------------+----------------------+----------------------+----------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+----------------------+------------------+------------------+----------------------------+------------------------+------------------------+------------------------+------------------------------+--------------------+--------------------------+--------------------+--------------------------+-----------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+-------------------+---------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-----------------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------------+--------------+--------------+-----------------+-------------+-------------+-----------------+-------------+-------------+-----------------+-------------+-------------+--------------+----------+----------+----------------+------------+------------+-------------+---------+---------+-------------+---------+---------+----------------------+---------------------+---------------------+---------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------------+----------------------+----------------------+----------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+----------------------+------------------+------------------+----------------------------+------------------------+------------------------+------------------------+------------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+------------------+--------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+--------------------+----------------+----------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+------------------+--------------+--------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------------------+----------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-------------------+---------------+---------------+------------------+--------------+--------------+------------------+--------------+--------------+------------------+--------------+--------------+---------------+-----------+-----------+-----------------+-------------+-------------+--------------+----------+----------+--------------+----------+----------+-----------------------+-------------------+-------------------+-----------------------------+-------------------------+-------------------------+-------------------------+-------------------------------+---------------------+---------------------------+---------------------+---------------------------+
# |user_id|merchant_id|label|gender|age|item_id|cat_id|brand_id|u_ttl_clk_atc_cnt|u_ttl_buy_cnt|u_ttl_atf_cnt|u_clk_atc_cnt_m5|u_clk_atc_cnt_m6|u_clk_atc_cnt_m7|u_clk_atc_cnt_m8|u_clk_atc_cnt_m9|u_clk_atc_cnt_m10|u_clk_atc_cnt_m11|u_buy_cnt_m5|u_buy_cnt_m6|u_buy_cnt_m7|u_buy_cnt_m8|u_buy_cnt_m9|u_buy_cnt_m10|u_buy_cnt_m11|u_atf_cnt_m5|u_atf_cnt_m6|u_atf_cnt_m7|u_atf_cnt_m8|u_atf_cnt_m9|u_atf_cnt_m10|u_atf_cnt_m11|u_ttl_clk_atc_ratio|u_ttl_buy_ratio|u_ttl_atf_ratio|u_clk_atc_ratio_m5|u_clk_atc_ratio_m6|u_clk_atc_ratio_m7|u_clk_atc_ratio_m8|u_clk_atc_ratio_m9|u_clk_atc_ratio_m10|u_clk_atc_ratio_m11|u_buy_ratio_m5|u_buy_ratio_m6|u_buy_ratio_m7|u_buy_ratio_m8|u_buy_ratio_m9|u_buy_ratio_m10|u_buy_ratio_m11|u_atf_ratio_m5|u_atf_ratio_m6|u_atf_ratio_m7|u_atf_ratio_m8|u_atf_ratio_m9|u_atf_ratio_m10|u_atf_ratio_m11|u_clk_atc_day_cnt|u_buy_day_cnt|u_atf_day_cnt|u_clk_atc_day_cnt_m5|u_clk_atc_day_cnt_m6|u_clk_atc_day_cnt_m7|u_clk_atc_day_cnt_m8|u_clk_atc_day_cnt_m9|u_clk_atc_day_cnt_m10|u_clk_atc_day_cnt_m11|u_buy_day_cnt_m5|u_buy_day_cnt_m6|u_buy_day_cnt_m7|u_buy_day_cnt_m8|u_buy_day_cnt_m9|u_buy_day_cnt_m10|u_buy_day_cnt_m11|u_atf_day_cnt_m5|u_atf_day_cnt_m6|u_atf_day_cnt_m7|u_atf_day_cnt_m8|u_atf_day_cnt_m9|u_atf_day_cnt_m10|u_atf_day_cnt_m11|u_clk_atc_item_div|u_buy_item_div|u_atf_item_div|u_clk_atc_cat_div|u_buy_cat_div|u_atf_cat_div|u_clk_atc_brd_div|u_buy_brd_div|u_atf_brd_div|u_clk_atc_mer_div|u_buy_mer_div|u_atf_mer_div|u_clk_atc_mean|u_buy_mean|u_atf_mean|u_clk_atc_median|u_buy_median|u_atf_median|u_clk_atc_std|u_buy_std|u_atf_std|u_clk_atc_max|u_buy_max|u_atf_max|u_clk_atc_day_cnt_mean|u_clk_atc_day_cnt_max|u_clk_atc_day_cnt_std|u_clk_atc_day_cnt_med|u_buy_day_cnt_mean|u_buy_day_cnt_max|u_buy_day_cnt_std|u_buy_day_cnt_med|u_atf_day_cnt_mean|u_atf_day_cnt_max|u_atf_day_cnt_std|u_atf_day_cnt_med|u_clk_atc_item_cnt_mean|u_clk_atc_item_cnt_max|u_clk_atc_item_cnt_std|u_clk_atc_item_cnt_med|u_buy_item_cnt_mean|u_buy_item_cnt_max|u_buy_item_cnt_std|u_buy_item_cnt_med|u_atf_item_cnt_mean|u_atf_item_cnt_max|u_atf_item_cnt_std|u_atf_item_cnt_med|u_clk_atc_cnt_double11|u_buy_cnt_double11|u_atf_cnt_double11|u_clk_atc_cnt_1wpre_double11|u_buy_cnt_1wpre_double11|u_atf_cnt_1wpre_double11|u_clk_atc_ratio_double11|u_clk_atc_ratio_1wpre_double11|u_buy_ratio_double11|u_buy_ratio_1wpre_double11|u_atf_ratio_double11|u_atf_ratio_1wpre_double11|m_ttl_clk_atc_cnt|m_ttl_buy_cnt|m_ttl_atf_cnt|m_clk_atc_cnt_m5|m_clk_atc_cnt_m6|m_clk_atc_cnt_m7|m_clk_atc_cnt_m8|m_clk_atc_cnt_m9|m_clk_atc_cnt_m10|m_clk_atc_cnt_m11|m_buy_cnt_m5|m_buy_cnt_m6|m_buy_cnt_m7|m_buy_cnt_m8|m_buy_cnt_m9|m_buy_cnt_m10|m_buy_cnt_m11|m_atf_cnt_m5|m_atf_cnt_m6|m_atf_cnt_m7|m_atf_cnt_m8|m_atf_cnt_m9|m_atf_cnt_m10|m_atf_cnt_m11|m_ttl_clk_atc_ratio|m_ttl_buy_ratio|m_ttl_atf_ratio|m_clk_atc_ratio_m5|m_clk_atc_ratio_m6|m_clk_atc_ratio_m7|m_clk_atc_ratio_m8|m_clk_atc_ratio_m9|m_clk_atc_ratio_m10|m_clk_atc_ratio_m11|m_buy_ratio_m5|m_buy_ratio_m6|m_buy_ratio_m7|m_buy_ratio_m8|m_buy_ratio_m9|m_buy_ratio_m10|m_buy_ratio_m11|m_atf_ratio_m5|m_atf_ratio_m6|m_atf_ratio_m7|m_atf_ratio_m8|m_atf_ratio_m9|m_atf_ratio_m10|m_atf_ratio_m11|m_clk_atc_day_cnt|m_buy_day_cnt|m_atf_day_cnt|m_clk_atc_day_cnt_m5|m_clk_atc_day_cnt_m6|m_clk_atc_day_cnt_m7|m_clk_atc_day_cnt_m8|m_clk_atc_day_cnt_m9|m_clk_atc_day_cnt_m10|m_clk_atc_day_cnt_m11|m_buy_day_cnt_m5|m_buy_day_cnt_m6|m_buy_day_cnt_m7|m_buy_day_cnt_m8|m_buy_day_cnt_m9|m_buy_day_cnt_m10|m_buy_day_cnt_m11|m_atf_day_cnt_m5|m_atf_day_cnt_m6|m_atf_day_cnt_m7|m_atf_day_cnt_m8|m_atf_day_cnt_m9|m_atf_day_cnt_m10|m_atf_day_cnt_m11|m_clk_atc_item_div|m_buy_item_div|m_atf_item_div|m_clk_atc_cat_div|m_buy_cat_div|m_atf_cat_div|m_clk_atc_brd_div|m_buy_brd_div|m_atf_brd_div|m_clk_atc_mer_div|m_buy_mer_div|m_atf_mer_div|m_clk_atc_mean|m_buy_mean|m_atf_mean|m_clk_atc_median|m_buy_median|m_atf_median|m_clk_atc_std|m_buy_std|m_atf_std|m_clk_atc_max|m_buy_max|m_atf_max|m_clk_atc_day_cnt_mean|m_clk_atc_day_cnt_max|m_clk_atc_day_cnt_std|m_clk_atc_day_cnt_med|m_buy_day_cnt_mean|m_buy_day_cnt_max|m_buy_day_cnt_std|m_buy_day_cnt_med|m_atf_day_cnt_mean|m_atf_day_cnt_max|m_atf_day_cnt_std|m_atf_day_cnt_med|m_clk_atc_item_cnt_mean|m_clk_atc_item_cnt_max|m_clk_atc_item_cnt_std|m_clk_atc_item_cnt_med|m_buy_item_cnt_mean|m_buy_item_cnt_max|m_buy_item_cnt_std|m_buy_item_cnt_med|m_atf_item_cnt_mean|m_atf_item_cnt_max|m_atf_item_cnt_std|m_atf_item_cnt_med|m_clk_atc_cnt_double11|m_buy_cnt_double11|m_atf_cnt_double11|m_clk_atc_cnt_1wpre_double11|m_buy_cnt_1wpre_double11|m_atf_cnt_1wpre_double11|m_clk_atc_ratio_double11|m_clk_atc_ratio_1wpre_double11|m_buy_ratio_double11|m_buy_ratio_1wpre_double11|m_atf_ratio_double11|m_atf_ratio_1wpre_double11|m_rpt_byr_num|m_rpt_byr_day|m_popular_age|m_popular_gender|um_ttl_clk_atc_cnt|um_ttl_buy_cnt|um_ttl_atf_cnt|um_clk_atc_cnt_m5|um_clk_atc_cnt_m6|um_clk_atc_cnt_m7|um_clk_atc_cnt_m8|um_clk_atc_cnt_m9|um_clk_atc_cnt_m10|um_clk_atc_cnt_m11|um_buy_cnt_m5|um_buy_cnt_m6|um_buy_cnt_m7|um_buy_cnt_m8|um_buy_cnt_m9|um_buy_cnt_m10|um_buy_cnt_m11|um_atf_cnt_m5|um_atf_cnt_m6|um_atf_cnt_m7|um_atf_cnt_m8|um_atf_cnt_m9|um_atf_cnt_m10|um_atf_cnt_m11|um_ttl_clk_atc_ratio|um_ttl_buy_ratio|um_ttl_atf_ratio|um_clk_atc_ratio_m5|um_clk_atc_ratio_m6|um_clk_atc_ratio_m7|um_clk_atc_ratio_m8|um_clk_atc_ratio_m9|um_clk_atc_ratio_m10|um_clk_atc_ratio_m11|um_buy_ratio_m5|um_buy_ratio_m6|um_buy_ratio_m7|um_buy_ratio_m8|um_buy_ratio_m9|um_buy_ratio_m10|um_buy_ratio_m11|um_atf_ratio_m5|um_atf_ratio_m6|um_atf_ratio_m7|um_atf_ratio_m8|um_atf_ratio_m9|um_atf_ratio_m10|um_atf_ratio_m11|um_clk_atc_day_cnt|um_buy_day_cnt|um_atf_day_cnt|um_clk_atc_day_cnt_m5|um_clk_atc_day_cnt_m6|um_clk_atc_day_cnt_m7|um_clk_atc_day_cnt_m8|um_clk_atc_day_cnt_m9|um_clk_atc_day_cnt_m10|um_clk_atc_day_cnt_m11|um_buy_day_cnt_m5|um_buy_day_cnt_m6|um_buy_day_cnt_m7|um_buy_day_cnt_m8|um_buy_day_cnt_m9|um_buy_day_cnt_m10|um_buy_day_cnt_m11|um_atf_day_cnt_m5|um_atf_day_cnt_m6|um_atf_day_cnt_m7|um_atf_day_cnt_m8|um_atf_day_cnt_m9|um_atf_day_cnt_m10|um_atf_day_cnt_m11|um_clk_atc_item_div|um_buy_item_div|um_atf_item_div|um_clk_atc_cat_div|um_buy_cat_div|um_atf_cat_div|um_clk_atc_brd_div|um_buy_brd_div|um_atf_brd_div|um_clk_atc_mer_div|um_buy_mer_div|um_atf_mer_div|um_clk_atc_mean|um_buy_mean|um_atf_mean|um_clk_atc_median|um_buy_median|um_atf_median|um_clk_atc_std|um_buy_std|um_atf_std|um_clk_atc_max|um_buy_max|um_atf_max|um_clk_atc_cnt_double11|um_buy_cnt_double11|um_atf_cnt_double11|um_clk_atc_cnt_1wpre_double11|um_buy_cnt_1wpre_double11|um_atf_cnt_1wpre_double11|um_clk_atc_ratio_double11|um_clk_atc_ratio_1wpre_double11|um_buy_ratio_double11|um_buy_ratio_1wpre_double11|um_atf_ratio_double11|um_atf_ratio_1wpre_double11|
# +-------+-----------+-----+------+---+-------+------+--------+-----------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+-------------------+---------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-----------------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------------+--------------+--------------+-----------------+-------------+-------------+-----------------+-------------+-------------+-----------------+-------------+-------------+--------------+----------+----------+----------------+------------+------------+-------------+---------+---------+-------------+---------+---------+----------------------+---------------------+---------------------+---------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------------+----------------------+----------------------+----------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+----------------------+------------------+------------------+----------------------------+------------------------+------------------------+------------------------+------------------------------+--------------------+--------------------------+--------------------+--------------------------+-----------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+-------------------+---------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-----------------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------------+--------------+--------------+-----------------+-------------+-------------+-----------------+-------------+-------------+-----------------+-------------+-------------+--------------+----------+----------+----------------+------------+------------+-------------+---------+---------+-------------+---------+---------+----------------------+---------------------+---------------------+---------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------------+----------------------+----------------------+----------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+----------------------+------------------+------------------+----------------------------+------------------------+------------------------+------------------------+------------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+------------------+--------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+--------------------+----------------+----------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+------------------+--------------+--------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------------------+----------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-------------------+---------------+---------------+------------------+--------------+--------------+------------------+--------------+--------------+------------------+--------------+--------------+---------------+-----------+-----------+-----------------+-------------+-------------+--------------+----------+----------+--------------+----------+----------+-----------------------+-------------------+-------------------+-----------------------------+-------------------------+-------------------------+-------------------------+-------------------------------+---------------------+---------------------------+---------------------+---------------------------+
# |      1|       1019|    1|     1|  3|1110495|   992|    6805|               27|            6|            0|               0|               0|               0|               0|               0|               14|               13|           0|           0|           0|           0|           0|            2|            4|           0|           0|           0|           0|           0|            0|            0|               0.82|           0.18|            0.0|               0.0|               0.0|               0.0|               0.0|               0.0|               0.88|               0.76|           0.0|           0.0|           0.0|           0.0|           0.0|           0.13|           0.24|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|                5|            3|            0|                   0|                   0|                   0|                   0|                   0|                    4|                    1|               0|               0|               0|               0|               0|                2|                1|               0|               0|               0|               0|               0|                0|                0|                12|             3|             0|                6|            3|            0|                9|            3|            0|                9|            3|            0|          2.33|      0.33|       0.0|             0.0|         0.0|         0.0|         5.72|     0.82|      0.0|           14|        2|        0|                  1.11|                    2|                 0.33|                  1.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|                   1.33|                     4|                   1.0|                   1.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                    13|                 4|                 0|                           0|                       0|                       0|                    0.76|                           0.0|                0.24|                       0.0|                 0.0|                       0.0|             6008|          941|          484|             463|             678|             460|             512|             727|             1265|             1903|         182|          83|          38|          67|          96|          180|          295|          85|          53|          32|          35|          48|          127|          104|               0.81|           0.13|           0.07|              0.63|              0.83|              0.87|              0.83|              0.83|                0.8|               0.83|          0.25|           0.1|          0.07|          0.11|          0.11|           0.11|           0.13|          0.12|          0.07|          0.06|          0.06|          0.06|           0.08|           0.05|              174|          139|          142|                  12|                  30|                  29|                  31|                  30|                   31|                   11|              20|              20|              15|              20|              24|               29|               11|              21|              23|              17|              20|              20|               31|               10|               147|            61|            60|                8|            8|            8|                1|            1|            1|                1|            1|            1|        684.17|    107.67|     63.33|           595.0|        89.5|        50.5|       305.96|    60.02|    36.47|         1265|      182|      127|                  1.16|                   10|                 0.57|                  1.0|              1.05|                3|             0.23|              1.0|              1.04|                3|             0.21|              1.0|                   1.33|                    14|                  0.99|                   1.0|               1.16|                 4|              0.45|               1.0|               1.07|                 6|              0.37|               1.0|                  1168|               234|                37|                         544|                      40|                      51|                    0.81|                          0.86|                0.16|                      0.06|                0.03|                      0.08|          191|          215|            3|               0|                10|             4|             0|                0|                0|                0|                0|                0|                 0|                10|            0|            0|            0|            0|            0|             0|             4|            0|            0|            0|            0|            0|             0|             0|                0.71|            0.29|             0.0|                0.0|                0.0|                0.0|                0.0|                0.0|                 0.0|                0.71|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.29|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|                 1|             1|             0|                    0|                    0|                    0|                    0|                    0|                     0|                     1|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                  1|              1|              0|                 1|             1|             0|                 1|             1|             0|                 1|             1|             0|            0.0|        0.0|        0.0|              0.0|          0.0|          0.0|           0.0|       0.0|       0.0|             0|         0|         0|                     10|                  4|                  0|                            0|                        0|                        0|                     0.71|                            0.0|                 0.29|                        0.0|                  0.0|                        0.0|
# |   1000|       3819|    0|     1|  2| 517962|     2|    8055|               66|            7|            6|               0|               0|               0|               0|               0|                0|               66|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|               0.84|           0.09|           0.08|               0.0|               0.0|               0.0|               0.0|               0.0|                0.0|               0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|                2|            1|            1|                   0|                   0|                   0|                   0|                   0|                    0|                    2|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|                21|             7|             6|                5|            4|            4|                9|            5|            3|                9|            5|            3|           0.0|       0.0|       0.0|             0.0|         0.0|         0.0|          0.0|      0.0|      0.0|            0|        0|        0|                  1.56|                    2|                 0.53|                  2.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|                   2.33|                    11|                  3.28|                   1.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                    18|                 7|                 0|                          48|                       0|                       6|                    0.72|                          0.89|                0.28|                       0.0|                 0.0|                      0.11|             6665|          467|          335|              15|              86|             102|             253|             385|              814|             5010|           0|           2|           3|           9|          11|           21|          421|           4|           7|           7|          17|          26|           73|          201|               0.89|           0.06|           0.04|              0.79|              0.91|              0.91|              0.91|              0.91|                0.9|               0.89|           0.0|          0.02|          0.03|          0.03|          0.03|           0.02|           0.07|          0.21|          0.07|          0.06|          0.06|          0.06|           0.08|           0.04|              153|           37|           75|                   7|                  22|                  22|                  30|                  30|                   31|                   11|               0|               2|               2|               5|               8|               12|                8|               3|               5|               6|              12|              14|               24|               11|                95|            40|            38|               14|           12|           11|                1|            1|            1|                1|            1|            1|        275.83|      7.67|     22.33|           177.5|         6.0|        12.0|       295.42|     7.79|    26.14|          814|       21|       73|                  1.26|                   13|                 0.76|                  1.0|              1.02|                3|             0.16|              1.0|               1.1|                3|             0.32|              1.0|                   1.88|                    29|                  2.09|                   1.0|               1.21|                 4|              0.53|               1.0|               1.25|                 5|              0.63|               1.0|                  2582|               408|                41|                        2172|                      11|                     142|                    0.85|                          0.93|                0.13|                       0.0|                0.01|                      0.06|           65|           73|            3|               1|                 5|             1|             0|                0|                0|                0|                0|                0|                 0|                 5|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|                0.83|            0.17|             0.0|                0.0|                0.0|                0.0|                0.0|                0.0|                 0.0|                0.83|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.17|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|                 2|             1|             0|                    0|                    0|                    0|                    0|                    0|                     0|                     2|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                  2|              1|              0|                 2|             1|             0|                 1|             1|             0|                 1|             1|             0|            0.0|        0.0|        0.0|              0.0|          0.0|          0.0|           0.0|       0.0|       0.0|             0|         0|         0|                      1|                  1|                  0|                            4|                        0|                        0|                      0.5|                            1.0|                  0.5|                        0.0|                  0.0|                        0.0|
# |   1000|        598|    0|     1|  2| 708788|   300|    6983|               66|            7|            6|               0|               0|               0|               0|               0|                0|               66|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|               0.84|           0.09|           0.08|               0.0|               0.0|               0.0|               0.0|               0.0|                0.0|               0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|                2|            1|            1|                   0|                   0|                   0|                   0|                   0|                    0|                    2|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|                21|             7|             6|                5|            4|            4|                9|            5|            3|                9|            5|            3|           0.0|       0.0|       0.0|             0.0|         0.0|         0.0|          0.0|      0.0|      0.0|            0|        0|        0|                  1.56|                    2|                 0.53|                  2.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|                   2.33|                    11|                  3.28|                   1.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                    18|                 7|                 0|                          48|                       0|                       6|                    0.72|                          0.89|                0.28|                       0.0|                 0.0|                      0.11|            82939|         4727|         4229|            3658|            9474|            4963|            4575|            7031|            13408|            39830|         174|         285|         178|         118|         258|          301|         3413|         333|         452|         280|         206|         351|          958|         1649|                0.9|           0.05|           0.05|              0.88|              0.93|              0.92|              0.93|              0.92|               0.91|               0.89|          0.04|          0.03|          0.03|          0.02|          0.03|           0.02|           0.08|          0.08|          0.04|          0.05|          0.04|          0.05|           0.07|           0.04|              178|          174|          185|                  14|                  30|                  31|                  31|                  30|                   31|                   11|              21|              29|              30|              25|              29|               31|                9|              21|              30|              31|              31|              30|               31|               11|               728|           370|           462|               12|            8|           10|                1|            1|            1|                1|            1|            1|       7184.83|     219.0|     430.0|          5997.0|       218.0|       342.0|      3692.64|    72.81|   271.14|        13408|      301|      958|                  1.69|                   48|                 1.69|                  1.0|              1.05|                6|             0.26|              1.0|              1.27|               12|             0.73|              1.0|                    3.1|                   124|                  4.77|                   1.0|               1.13|                 8|              0.46|               1.0|               1.61|                25|              1.48|               1.0|                 20563|              3393|               310|                       16156|                      13|                    1064|                    0.85|                          0.94|                0.14|                       0.0|                0.01|                      0.06|          528|          721|            3|               1|                 3|             1|             0|                0|                0|                0|                0|                0|                 0|                 3|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|                0.75|            0.25|             0.0|                0.0|                0.0|                0.0|                0.0|                0.0|                 0.0|                0.75|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.25|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|                 2|             1|             0|                    0|                    0|                    0|                    0|                    0|                     0|                     2|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                  1|              1|              0|                 1|             1|             0|                 1|             1|             0|                 1|             1|             0|            0.0|        0.0|        0.0|              0.0|          0.0|          0.0|           0.0|       0.0|       0.0|             0|         0|         0|                      1|                  1|                  0|                            2|                        0|                        0|                      0.5|                            1.0|                  0.5|                        0.0|                  0.0|                        0.0|
# | 100001|       1963|    0|     1|  0|1070887|  1023|    6109|               87|            3|            0|               0|               0|               0|               1|               1|                1|               84|           0|           0|           0|           0|           0|            0|            3|           0|           0|           0|           0|           0|            0|            0|               0.97|           0.03|            0.0|               0.0|               0.0|               0.0|               1.0|               1.0|                1.0|               0.97|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.03|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|                4|            1|            0|                   0|                   0|                   0|                   1|                   1|                    1|                    1|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                0|                74|             3|             0|               20|            3|            0|               35|            3|            0|               40|            3|            0|           0.5|       0.0|       0.0|             0.5|         0.0|         0.0|         0.55|      0.0|      0.0|            1|        0|        0|                   1.0|                    1|                  0.0|                  1.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|                   1.85|                    10|                  1.97|                   1.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                    84|                 3|                 0|                           0|                       0|                       0|                    0.97|                           0.0|                0.03|                       0.0|                 0.0|                       0.0|            57375|         1497|         1553|            2393|            5932|            1472|            1859|            2207|             5337|            38175|          89|          81|          10|          16|          18|           13|         1270|         126|         204|          59|          51|          64|          309|          740|               0.95|           0.02|           0.03|              0.92|              0.95|              0.96|              0.97|              0.96|               0.94|               0.95|          0.03|          0.01|          0.01|          0.01|          0.01|            0.0|           0.03|          0.05|          0.03|          0.04|          0.03|          0.03|           0.05|           0.02|              176|           77|          153|                  12|                  30|                  31|                  31|                  30|                   31|                   11|              18|              18|               8|               8|              13|               11|                1|              20|              28|              19|              21|              25|               29|               11|               500|           190|           324|               28|           17|           24|                1|            1|            1|                1|            1|            1|        3200.0|     37.83|     135.5|          2300.0|        17.0|        95.0|      1921.05|    36.72|   102.95|         5932|       89|      309|                  1.54|                   35|                 1.42|                  1.0|              1.02|                3|             0.13|              1.0|              1.27|               10|             0.75|              1.0|                   3.75|                   119|                  6.05|                   2.0|               1.21|                 7|              0.57|               1.0|               1.81|                22|              1.75|               1.0|                 20991|              1270|               102|                       13768|                       0|                     494|                    0.94|                          0.97|                0.06|                       0.0|                 0.0|                      0.03|          205|          224|            3|               1|                 6|             1|             0|                0|                0|                0|                0|                0|                 0|                 6|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|                0.86|            0.14|             0.0|                0.0|                0.0|                0.0|                0.0|                0.0|                 0.0|                0.86|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.14|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|                 1|             1|             0|                    0|                    0|                    0|                    0|                    0|                     0|                     1|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                  4|              1|              0|                 1|             1|             0|                 1|             1|             0|                 1|             1|             0|            0.0|        0.0|        0.0|              0.0|          0.0|          0.0|           0.0|       0.0|       0.0|             0|         0|         0|                      6|                  1|                  0|                            0|                        0|                        0|                     0.86|                            0.0|                 0.14|                        0.0|                  0.0|                        0.0|
# | 100001|       2273|    0|     1|  0| 191499|  1421|    2240|               87|            3|            0|               0|               0|               0|               1|               1|                1|               84|           0|           0|           0|           0|           0|            0|            3|           0|           0|           0|           0|           0|            0|            0|               0.97|           0.03|            0.0|               0.0|               0.0|               0.0|               1.0|               1.0|                1.0|               0.97|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.03|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|                4|            1|            0|                   0|                   0|                   0|                   1|                   1|                    1|                    1|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                0|                74|             3|             0|               20|            3|            0|               35|            3|            0|               40|            3|            0|           0.5|       0.0|       0.0|             0.5|         0.0|         0.0|         0.55|      0.0|      0.0|            1|        0|        0|                   1.0|                    1|                  0.0|                  1.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|                   1.85|                    10|                  1.97|                   1.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                    84|                 3|                 0|                           0|                       0|                       0|                    0.97|                           0.0|                0.03|                       0.0|                 0.0|                       0.0|            37573|         4445|         2947|             706|            2204|            1755|            1802|            3362|             3248|            24496|         322|         159|         162|         141|         268|          179|         3214|         154|         120|         159|         138|         317|          264|         1795|               0.84|            0.1|           0.07|               0.6|              0.89|              0.85|              0.87|              0.85|               0.88|               0.83|          0.27|          0.06|          0.08|          0.07|          0.07|           0.05|           0.11|          0.13|          0.05|          0.08|          0.07|          0.08|           0.07|           0.06|              176|          181|          174|                  12|                  30|                  31|                  31|                  30|                   31|                   11|              20|              30|              29|              30|              30|               31|               11|              20|              28|              27|              28|              29|               31|               11|                95|            46|            44|               14|            8|            5|                1|            1|            1|                1|            1|            1|        2179.5|    205.17|     192.0|          2003.0|       170.5|       156.5|      1003.44|    72.66|     79.3|         3362|      322|      317|                  1.49|                   20|                  1.1|                  1.0|              1.01|                6|             0.15|              1.0|              1.18|                6|             0.51|              1.0|                    1.8|                    21|                  1.47|                   1.0|               1.02|                 3|              0.14|               1.0|               1.19|                 6|              0.54|               1.0|                 14651|              3194|               268|                        8666|                      16|                    1297|                    0.81|                          0.87|                0.18|                       0.0|                0.01|                      0.13|          238|          298|            3|               0|                 1|             1|             0|                0|                0|                0|                0|                0|                 0|                 1|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|                 0.5|             0.5|             0.0|                0.0|                0.0|                0.0|                0.0|                0.0|                 0.0|                 0.5|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.5|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|                 1|             1|             0|                    0|                    0|                    0|                    0|                    0|                     0|                     1|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                  1|              1|              0|                 1|             1|             0|                 1|             1|             0|                 1|             1|             0|            0.0|        0.0|        0.0|              0.0|          0.0|          0.0|           0.0|       0.0|       0.0|             0|         0|         0|                      1|                  1|                  0|                            0|                        0|                        0|                      0.5|                            0.0|                  0.5|                        0.0|                  0.0|                        0.0|
# +-------+-----------+-----+------+---+-------+------+--------+-----------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+-------------------+---------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-----------------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------------+--------------+--------------+-----------------+-------------+-------------+-----------------+-------------+-------------+-----------------+-------------+-------------+--------------+----------+----------+----------------+------------+------------+-------------+---------+---------+-------------+---------+---------+----------------------+---------------------+---------------------+---------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------------+----------------------+----------------------+----------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+----------------------+------------------+------------------+----------------------------+------------------------+------------------------+------------------------+------------------------------+--------------------+--------------------------+--------------------+--------------------------+-----------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+-------------------+---------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-----------------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+------------------+--------------+--------------+-----------------+-------------+-------------+-----------------+-------------+-------------+-----------------+-------------+-------------+--------------+----------+----------+----------------+------------+------------+-------------+---------+---------+-------------+---------+---------+----------------------+---------------------+---------------------+---------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------------+----------------------+----------------------+----------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+----------------------+------------------+------------------+----------------------------+------------------------+------------------------+------------------------+------------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+------------------+--------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+--------------------+----------------+----------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+------------------+--------------+--------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------------------+----------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-------------------+---------------+---------------+------------------+--------------+--------------+------------------+--------------+--------------+------------------+--------------+--------------+---------------+-----------+-----------+-----------------+-------------+-------------+--------------+----------+----------+--------------+----------+----------+-----------------------+-------------------+-------------------+-----------------------------+-------------------------+-------------------------+-------------------------+-------------------------------+---------------------+---------------------------+---------------------+---------------------------+


In [ ]:
test_feature_4 = test_feature_3 \
                .join(brd_feature, "brand_id", "left") \
                .orderBy("user_id", "merchant_id")

# +--------+-------+-----------+-----+------+---+-------+------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+--------------+--------------+--------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+----------+----------+----------+----------+-------------------+-------------------+-------------------+-------------------+-------------------------+-------------------------+-------------------------+-------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+
# |brand_id|user_id|merchant_id|label|gender|age|item_id|cat_id|u_ttl_clk_cnt|u_ttl_atc_cnt|u_ttl_buy_cnt|u_ttl_atf_cnt|u_clk_cnt_m5|u_clk_cnt_m6|u_clk_cnt_m7|u_clk_cnt_m8|u_clk_cnt_m9|u_clk_cnt_m10|u_clk_cnt_m11|u_atc_cnt_m5|u_atc_cnt_m6|u_atc_cnt_m7|u_atc_cnt_m8|u_atc_cnt_m9|u_atc_cnt_m10|u_atc_cnt_m11|u_buy_cnt_m5|u_buy_cnt_m6|u_buy_cnt_m7|u_buy_cnt_m8|u_buy_cnt_m9|u_buy_cnt_m10|u_buy_cnt_m11|u_atf_cnt_m5|u_atf_cnt_m6|u_atf_cnt_m7|u_atf_cnt_m8|u_atf_cnt_m9|u_atf_cnt_m10|u_atf_cnt_m11|u_ttl_clk_ratio|u_ttl_atc_ratio|u_ttl_buy_ratio|u_ttl_atf_ratio|u_clk_ratio_m5|u_clk_ratio_m6|u_clk_ratio_m7|u_clk_ratio_m8|u_clk_ratio_m9|u_clk_ratio_m10|u_clk_ratio_m11|u_atc_ratio_m5|u_atc_ratio_m6|u_atc_ratio_m7|u_atc_ratio_m8|u_atc_ratio_m9|u_atc_ratio_m10|u_atc_ratio_m11|u_buy_ratio_m5|u_buy_ratio_m6|u_buy_ratio_m7|u_buy_ratio_m8|u_buy_ratio_m9|u_buy_ratio_m10|u_buy_ratio_m11|u_atf_ratio_m5|u_atf_ratio_m6|u_atf_ratio_m7|u_atf_ratio_m8|u_atf_ratio_m9|u_atf_ratio_m10|u_atf_ratio_m11|u_clk_day_cnt|u_atc_day_cnt|u_buy_day_cnt|u_atf_day_cnt|u_clk_day_cnt_m5|u_clk_day_cnt_m6|u_clk_day_cnt_m7|u_clk_day_cnt_m8|u_clk_day_cnt_m9|u_clk_day_cnt_m10|u_clk_day_cnt_m11|u_atc_day_cnt_m5|u_atc_day_cnt_m6|u_atc_day_cnt_m7|u_atc_day_cnt_m8|u_atc_day_cnt_m9|u_atc_day_cnt_m10|u_atc_day_cnt_m11|u_buy_day_cnt_m5|u_buy_day_cnt_m6|u_buy_day_cnt_m7|u_buy_day_cnt_m8|u_buy_day_cnt_m9|u_buy_day_cnt_m10|u_buy_day_cnt_m11|u_atf_day_cnt_m5|u_atf_day_cnt_m6|u_atf_day_cnt_m7|u_atf_day_cnt_m8|u_atf_day_cnt_m9|u_atf_day_cnt_m10|u_atf_day_cnt_m11|u_clk_item_div|u_atc_item_div|u_buy_item_div|u_atf_item_div|u_clk_cat_div|u_atc_cat_div|u_buy_cat_div|u_atf_cat_div|u_clk_brd_div|u_atc_brd_div|u_buy_brd_div|u_atf_brd_div|u_clk_mer_div|u_atc_mer_div|u_buy_mer_div|u_atf_mer_div|u_clk_mean|u_atc_mean|u_buy_mean|u_atf_mean|u_clk_median|u_atc_median|u_buy_median|u_atf_median|u_clk_std|u_atc_std|u_buy_std|u_atf_std|u_clk_max|u_atc_max|u_buy_max|u_atf_max|u_clk_day_cnt_mean|u_clk_day_cnt_max|u_clk_day_cnt_std|u_clk_day_cnt_med|u_atc_day_cnt_mean|u_atc_day_cnt_max|u_atc_day_cnt_std|u_atc_day_cnt_med|u_buy_day_cnt_mean|u_buy_day_cnt_max|u_buy_day_cnt_std|u_buy_day_cnt_med|u_atf_day_cnt_mean|u_atf_day_cnt_max|u_atf_day_cnt_std|u_atf_day_cnt_med|u_clk_item_cnt_mean|u_clk_item_cnt_max|u_clk_item_cnt_std|u_clk_item_cnt_med|u_atc_item_cnt_mean|u_atc_item_cnt_max|u_atc_item_cnt_std|u_atc_item_cnt_med|u_buy_item_cnt_mean|u_buy_item_cnt_max|u_buy_item_cnt_std|u_buy_item_cnt_med|u_atf_item_cnt_mean|u_atf_item_cnt_max|u_atf_item_cnt_std|u_atf_item_cnt_med|u_clk_cnt_double11|u_atc_cnt_double11|u_buy_cnt_double11|u_atf_cnt_double11|u_clk_cnt_1wpre_double11|u_atc_cnt_1wpre_double11|u_buy_cnt_1wpre_double11|u_atf_cnt_1wpre_double11|u_clk_ratio_double11|u_clk_ratio_1wpre_double11|u_atc_ratio_double11|u_atc_ratio_1wpre_double11|u_buy_ratio_double11|u_buy_ratio_1wpre_double11|u_atf_ratio_double11|u_atf_ratio_1wpre_double11|m_ttl_clk_cnt|m_ttl_atc_cnt|m_ttl_buy_cnt|m_ttl_atf_cnt|m_clk_cnt_m5|m_clk_cnt_m6|m_clk_cnt_m7|m_clk_cnt_m8|m_clk_cnt_m9|m_clk_cnt_m10|m_clk_cnt_m11|m_atc_cnt_m5|m_atc_cnt_m6|m_atc_cnt_m7|m_atc_cnt_m8|m_atc_cnt_m9|m_atc_cnt_m10|m_atc_cnt_m11|m_buy_cnt_m5|m_buy_cnt_m6|m_buy_cnt_m7|m_buy_cnt_m8|m_buy_cnt_m9|m_buy_cnt_m10|m_buy_cnt_m11|m_atf_cnt_m5|m_atf_cnt_m6|m_atf_cnt_m7|m_atf_cnt_m8|m_atf_cnt_m9|m_atf_cnt_m10|m_atf_cnt_m11|m_ttl_clk_ratio|m_ttl_atc_ratio|m_ttl_buy_ratio|m_ttl_atf_ratio|m_clk_ratio_m5|m_clk_ratio_m6|m_clk_ratio_m7|m_clk_ratio_m8|m_clk_ratio_m9|m_clk_ratio_m10|m_clk_ratio_m11|m_atc_ratio_m5|m_atc_ratio_m6|m_atc_ratio_m7|m_atc_ratio_m8|m_atc_ratio_m9|m_atc_ratio_m10|m_atc_ratio_m11|m_buy_ratio_m5|m_buy_ratio_m6|m_buy_ratio_m7|m_buy_ratio_m8|m_buy_ratio_m9|m_buy_ratio_m10|m_buy_ratio_m11|m_atf_ratio_m5|m_atf_ratio_m6|m_atf_ratio_m7|m_atf_ratio_m8|m_atf_ratio_m9|m_atf_ratio_m10|m_atf_ratio_m11|m_clk_day_cnt|m_atc_day_cnt|m_buy_day_cnt|m_atf_day_cnt|m_clk_day_cnt_m5|m_clk_day_cnt_m6|m_clk_day_cnt_m7|m_clk_day_cnt_m8|m_clk_day_cnt_m9|m_clk_day_cnt_m10|m_clk_day_cnt_m11|m_atc_day_cnt_m5|m_atc_day_cnt_m6|m_atc_day_cnt_m7|m_atc_day_cnt_m8|m_atc_day_cnt_m9|m_atc_day_cnt_m10|m_atc_day_cnt_m11|m_buy_day_cnt_m5|m_buy_day_cnt_m6|m_buy_day_cnt_m7|m_buy_day_cnt_m8|m_buy_day_cnt_m9|m_buy_day_cnt_m10|m_buy_day_cnt_m11|m_atf_day_cnt_m5|m_atf_day_cnt_m6|m_atf_day_cnt_m7|m_atf_day_cnt_m8|m_atf_day_cnt_m9|m_atf_day_cnt_m10|m_atf_day_cnt_m11|m_clk_item_div|m_atc_item_div|m_buy_item_div|m_atf_item_div|m_clk_cat_div|m_atc_cat_div|m_buy_cat_div|m_atf_cat_div|m_clk_brd_div|m_atc_brd_div|m_buy_brd_div|m_atf_brd_div|m_clk_mer_div|m_atc_mer_div|m_buy_mer_div|m_atf_mer_div|m_clk_mean|m_atc_mean|m_buy_mean|m_atf_mean|m_clk_median|m_atc_median|m_buy_median|m_atf_median|m_clk_std|m_atc_std|m_buy_std|m_atf_std|m_clk_max|m_atc_max|m_buy_max|m_atf_max|m_clk_day_cnt_mean|m_clk_day_cnt_max|m_clk_day_cnt_std|m_clk_day_cnt_med|m_atc_day_cnt_mean|m_atc_day_cnt_max|m_atc_day_cnt_std|m_atc_day_cnt_med|m_buy_day_cnt_mean|m_buy_day_cnt_max|m_buy_day_cnt_std|m_buy_day_cnt_med|m_atf_day_cnt_mean|m_atf_day_cnt_max|m_atf_day_cnt_std|m_atf_day_cnt_med|m_clk_item_cnt_mean|m_clk_item_cnt_max|m_clk_item_cnt_std|m_clk_item_cnt_med|m_atc_item_cnt_mean|m_atc_item_cnt_max|m_atc_item_cnt_std|m_atc_item_cnt_med|m_buy_item_cnt_mean|m_buy_item_cnt_max|m_buy_item_cnt_std|m_buy_item_cnt_med|m_atf_item_cnt_mean|m_atf_item_cnt_max|m_atf_item_cnt_std|m_atf_item_cnt_med|m_clk_cnt_double11|m_atc_cnt_double11|m_buy_cnt_double11|m_atf_cnt_double11|m_clk_cnt_1wpre_double11|m_atc_cnt_1wpre_double11|m_buy_cnt_1wpre_double11|m_atf_cnt_1wpre_double11|m_clk_ratio_double11|m_clk_ratio_1wpre_double11|m_atc_ratio_double11|m_atc_ratio_1wpre_double11|m_buy_ratio_double11|m_buy_ratio_1wpre_double11|m_atf_ratio_double11|m_atf_ratio_1wpre_double11|m_rpt_byr_num|m_rpt_byr_day|m_popular_age|m_popular_gender|um_ttl_clk_cnt|um_ttl_atc_cnt|um_ttl_buy_cnt|um_ttl_atf_cnt|um_clk_cnt_m5|um_clk_cnt_m6|um_clk_cnt_m7|um_clk_cnt_m8|um_clk_cnt_m9|um_clk_cnt_m10|um_clk_cnt_m11|um_atc_cnt_m5|um_atc_cnt_m6|um_atc_cnt_m7|um_atc_cnt_m8|um_atc_cnt_m9|um_atc_cnt_m10|um_atc_cnt_m11|um_buy_cnt_m5|um_buy_cnt_m6|um_buy_cnt_m7|um_buy_cnt_m8|um_buy_cnt_m9|um_buy_cnt_m10|um_buy_cnt_m11|um_atf_cnt_m5|um_atf_cnt_m6|um_atf_cnt_m7|um_atf_cnt_m8|um_atf_cnt_m9|um_atf_cnt_m10|um_atf_cnt_m11|um_ttl_clk_ratio|um_ttl_atc_ratio|um_ttl_buy_ratio|um_ttl_atf_ratio|um_clk_ratio_m5|um_clk_ratio_m6|um_clk_ratio_m7|um_clk_ratio_m8|um_clk_ratio_m9|um_clk_ratio_m10|um_clk_ratio_m11|um_atc_ratio_m5|um_atc_ratio_m6|um_atc_ratio_m7|um_atc_ratio_m8|um_atc_ratio_m9|um_atc_ratio_m10|um_atc_ratio_m11|um_buy_ratio_m5|um_buy_ratio_m6|um_buy_ratio_m7|um_buy_ratio_m8|um_buy_ratio_m9|um_buy_ratio_m10|um_buy_ratio_m11|um_atf_ratio_m5|um_atf_ratio_m6|um_atf_ratio_m7|um_atf_ratio_m8|um_atf_ratio_m9|um_atf_ratio_m10|um_atf_ratio_m11|um_clk_day_cnt|um_atc_day_cnt|um_buy_day_cnt|um_atf_day_cnt|um_clk_day_cnt_m5|um_clk_day_cnt_m6|um_clk_day_cnt_m7|um_clk_day_cnt_m8|um_clk_day_cnt_m9|um_clk_day_cnt_m10|um_clk_day_cnt_m11|um_atc_day_cnt_m5|um_atc_day_cnt_m6|um_atc_day_cnt_m7|um_atc_day_cnt_m8|um_atc_day_cnt_m9|um_atc_day_cnt_m10|um_atc_day_cnt_m11|um_buy_day_cnt_m5|um_buy_day_cnt_m6|um_buy_day_cnt_m7|um_buy_day_cnt_m8|um_buy_day_cnt_m9|um_buy_day_cnt_m10|um_buy_day_cnt_m11|um_atf_day_cnt_m5|um_atf_day_cnt_m6|um_atf_day_cnt_m7|um_atf_day_cnt_m8|um_atf_day_cnt_m9|um_atf_day_cnt_m10|um_atf_day_cnt_m11|um_clk_item_div|um_atc_item_div|um_buy_item_div|um_atf_item_div|um_clk_cat_div|um_atc_cat_div|um_buy_cat_div|um_atf_cat_div|um_clk_brd_div|um_atc_brd_div|um_buy_brd_div|um_atf_brd_div|um_clk_mer_div|um_atc_mer_div|um_buy_mer_div|um_atf_mer_div|um_clk_mean|um_atc_mean|um_buy_mean|um_atf_mean|um_clk_median|um_atc_median|um_buy_median|um_atf_median|um_clk_std|um_atc_std|um_buy_std|um_atf_std|um_clk_max|um_atc_max|um_buy_max|um_atf_max|um_clk_cnt_double11|um_atc_cnt_double11|um_buy_cnt_double11|um_atf_cnt_double11|um_clk_cnt_1wpre_double11|um_atc_cnt_1wpre_double11|um_buy_cnt_1wpre_double11|um_atf_cnt_1wpre_double11|um_clk_ratio_double11|um_clk_ratio_1wpre_double11|um_atc_ratio_double11|um_atc_ratio_1wpre_double11|um_buy_ratio_double11|um_buy_ratio_1wpre_double11|um_atf_ratio_double11|um_atf_ratio_1wpre_double11|b_ttl_clk_cnt|b_ttl_atc_cnt|b_ttl_buy_cnt|b_ttl_atf_cnt|b_clk_cnt_m5|b_clk_cnt_m6|b_clk_cnt_m7|b_clk_cnt_m8|b_clk_cnt_m9|b_clk_cnt_m10|b_clk_cnt_m11|b_atc_cnt_m5|b_atc_cnt_m6|b_atc_cnt_m7|b_atc_cnt_m8|b_atc_cnt_m9|b_atc_cnt_m10|b_atc_cnt_m11|b_buy_cnt_m5|b_buy_cnt_m6|b_buy_cnt_m7|b_buy_cnt_m8|b_buy_cnt_m9|b_buy_cnt_m10|b_buy_cnt_m11|b_atf_cnt_m5|b_atf_cnt_m6|b_atf_cnt_m7|b_atf_cnt_m8|b_atf_cnt_m9|b_atf_cnt_m10|b_atf_cnt_m11|b_ttl_clk_ratio|b_ttl_atc_ratio|b_ttl_buy_ratio|b_ttl_atf_ratio|b_clk_ratio_m5|b_clk_ratio_m6|b_clk_ratio_m7|b_clk_ratio_m8|b_clk_ratio_m9|b_clk_ratio_m10|b_clk_ratio_m11|b_atc_ratio_m5|b_atc_ratio_m6|b_atc_ratio_m7|b_atc_ratio_m8|b_atc_ratio_m9|b_atc_ratio_m10|b_atc_ratio_m11|b_buy_ratio_m5|b_buy_ratio_m6|b_buy_ratio_m7|b_buy_ratio_m8|b_buy_ratio_m9|b_buy_ratio_m10|b_buy_ratio_m11|b_atf_ratio_m5|b_atf_ratio_m6|b_atf_ratio_m7|b_atf_ratio_m8|b_atf_ratio_m9|b_atf_ratio_m10|b_atf_ratio_m11|b_clk_day_cnt|b_atc_day_cnt|b_buy_day_cnt|b_atf_day_cnt|b_clk_day_cnt_m5|b_clk_day_cnt_m6|b_clk_day_cnt_m7|b_clk_day_cnt_m8|b_clk_day_cnt_m9|b_clk_day_cnt_m10|b_clk_day_cnt_m11|b_atc_day_cnt_m5|b_atc_day_cnt_m6|b_atc_day_cnt_m7|b_atc_day_cnt_m8|b_atc_day_cnt_m9|b_atc_day_cnt_m10|b_atc_day_cnt_m11|b_buy_day_cnt_m5|b_buy_day_cnt_m6|b_buy_day_cnt_m7|b_buy_day_cnt_m8|b_buy_day_cnt_m9|b_buy_day_cnt_m10|b_buy_day_cnt_m11|b_atf_day_cnt_m5|b_atf_day_cnt_m6|b_atf_day_cnt_m7|b_atf_day_cnt_m8|b_atf_day_cnt_m9|b_atf_day_cnt_m10|b_atf_day_cnt_m11|b_clk_mean|b_atc_mean|b_buy_mean|b_atf_mean|b_clk_median|b_atc_median|b_buy_median|b_atf_median|b_clk_std|b_atc_std|b_buy_std|b_atf_std|b_clk_max|b_atc_max|b_buy_max|b_atf_max|b_clk_cnt_double11|b_atc_cnt_double11|b_buy_cnt_double11|b_atf_cnt_double11|b_clk_cnt_1wpre_double11|b_atc_cnt_1wpre_double11|b_buy_cnt_1wpre_double11|b_atf_cnt_1wpre_double11|b_clk_ratio_double11|b_clk_ratio_1wpre_double11|b_atc_ratio_double11|b_atc_ratio_1wpre_double11|b_buy_ratio_double11|b_buy_ratio_1wpre_double11|b_atf_ratio_double11|b_atf_ratio_1wpre_double11|b_rpt_byr_num|b_rpt_byr_day|b_popular_age|b_popular_gender|
# +--------+-------+-----------+-----+------+---+-------+------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+--------------+--------------+--------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+----------+----------+----------+----------+-------------------+-------------------+-------------------+-------------------+-------------------------+-------------------------+-------------------------+-------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+
# |    6805|      1|       1019|    1|     1|  3|1110495|   992|           27|            0|            6|            0|           0|           0|           0|           0|           0|           14|           13|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            2|            4|           0|           0|           0|           0|           0|            0|            0|           0.82|            0.0|           0.18|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|           0.88|           0.76|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|           0.13|           0.24|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            5|            0|            3|            0|               0|               0|               0|               0|               0|                4|                1|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                2|                1|               0|               0|               0|               0|               0|                0|                0|            12|             0|             3|             0|            6|            0|            3|            0|            9|            0|            3|            0|            9|            0|            3|            0|      2.33|       0.0|      0.33|       0.0|         0.0|         0.0|         0.0|         0.0|     5.72|      0.0|     0.82|      0.0|       14|        0|        2|        0|              1.11|                2|             0.33|              1.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.33|                 4|               1.0|               1.0|                0.0|                 0|               0.0|               0.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                13|                 0|                 4|                 0|                       0|                       0|                       0|                       0|                0.76|                       0.0|                 0.0|                       0.0|                0.24|                       0.0|                 0.0|                       0.0|         6003|            5|          941|          484|         463|         678|         459|         511|         727|         1263|         1902|           0|           0|           1|           1|           0|            2|            1|         182|          83|          38|          67|          96|          180|          295|          85|          53|          32|          35|          48|          127|          104|           0.81|            0.0|           0.13|           0.07|          0.63|          0.83|          0.87|          0.83|          0.83|            0.8|           0.83|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.25|           0.1|          0.07|          0.11|          0.11|           0.11|           0.13|          0.12|          0.07|          0.06|          0.06|          0.06|           0.08|           0.05|          174|            5|          139|          142|              12|              30|              29|              31|              30|               31|               11|               0|               0|               1|               1|               0|                2|                1|              20|              20|              15|              20|              24|               29|               11|              21|              23|              17|              20|              20|               31|               10|           147|             3|            61|            60|            8|            2|            8|            8|            1|            0|            1|            1|            1|            1|            1|            1|     683.5|      0.67|    107.67|     63.33|       594.5|         0.5|        89.5|        50.5|   305.46|     0.82|    60.02|    36.47|     1263|        2|      182|      127|              1.17|               10|             0.57|              1.0|               1.0|                1|              0.0|              1.0|              1.05|                3|             0.23|              1.0|              1.04|                3|             0.21|              1.0|               1.33|                14|              0.99|               1.0|                1.0|                 1|               0.0|               1.0|               1.16|                 4|              0.45|               1.0|               1.07|                 6|              0.37|               1.0|              1167|                 1|               234|                37|                     544|                       0|                      40|                      51|                0.81|                      0.86|                 0.0|                       0.0|                0.16|                      0.06|                0.03|                      0.08|          191|          215|            3|               0|            10|             0|             4|             0|            0|            0|            0|            0|            0|             0|            10|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             4|            0|            0|            0|            0|            0|             0|             0|            0.71|             0.0|            0.29|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.71|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.29|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             1|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              1|              0|              1|              0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                 10|                  0|                  4|                  0|                        0|                        0|                        0|                        0|                 0.71|                        0.0|                  0.0|                        0.0|                 0.29|                        0.0|                  0.0|                        0.0|         6003|            0|          941|          484|         463|         678|         459|         511|         727|         1263|         1902|           0|           0|           0|           0|           0|            0|            0|         182|          83|          38|          67|          96|          180|          295|          85|          53|          32|          35|          48|          127|          104|           0.81|            0.0|           0.13|           0.07|          0.63|          0.83|          0.87|          0.83|          0.83|            0.8|           0.83|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.25|           0.1|          0.07|          0.11|          0.11|           0.11|           0.13|          0.12|          0.07|          0.06|          0.06|          0.06|           0.08|           0.05|          174|            0|          139|          142|              12|              30|              29|              31|              30|               31|               11|               0|               0|               0|               0|               0|                0|                0|              20|              20|              15|              20|              24|               29|               11|              21|              23|              17|              20|              20|               31|               10|     683.5|       0.0|    107.67|     63.33|       594.5|         0.0|        89.5|        50.5|   305.46|      0.0|    60.02|    36.47|     1263|        0|      182|      127|              1167|                 0|               234|                37|                     544|                       0|                      40|                      51|                0.81|                      0.86|                 0.0|                       0.0|                0.16|                      0.06|                0.03|                      0.08|          191|          215|            3|               0|
# |    8055|   1000|       3819|    0|     1|  2| 517962|     2|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|         6642|           23|          467|          335|          15|          86|         102|         253|         384|          813|         4989|           0|           0|           0|           0|           1|            1|           21|           0|           2|           3|           9|          11|           21|          421|           4|           7|           7|          17|          26|           73|          201|           0.89|            0.0|           0.06|           0.04|          0.79|          0.91|          0.91|          0.91|          0.91|            0.9|           0.89|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|          0.02|          0.03|          0.03|          0.03|           0.02|           0.07|          0.21|          0.07|          0.06|          0.06|          0.06|           0.08|           0.04|          153|            8|           37|           75|               7|              22|              22|              30|              30|               31|               11|               0|               0|               0|               0|               1|                1|                6|               0|               2|               2|               5|               8|               12|                8|               3|               5|               6|              12|              14|               24|               11|            95|            12|            40|            38|           14|            6|           12|           11|            1|            0|            1|            1|            1|            1|            1|            1|     275.5|      0.33|      7.67|     22.33|       177.5|         0.0|         6.0|        12.0|   294.98|     0.52|     7.79|    26.14|      813|        1|       21|       73|              1.26|               13|             0.76|              1.0|              1.05|                2|             0.22|              1.0|              1.02|                3|             0.16|              1.0|               1.1|                3|             0.32|              1.0|               1.88|                29|              2.09|               1.0|                1.1|                 2|              0.31|               1.0|               1.21|                 4|              0.53|               1.0|               1.25|                 5|              0.63|               1.0|              2575|                 7|               408|                41|                    2160|                      12|                      11|                     142|                0.85|                      0.93|                 0.0|                      0.01|                0.13|                       0.0|                0.01|                      0.06|           65|           73|            3|               1|             5|             0|             1|             0|            0|            0|            0|            0|            0|             0|             5|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|            0.83|             0.0|            0.17|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.83|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.17|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             2|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 2|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              2|              0|              1|              0|             2|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                  1|                  0|                  1|                  0|                        4|                        0|                        0|                        0|                  0.5|                        1.0|                  0.0|                        0.0|                  0.5|                        0.0|                  0.0|                        0.0|         6642|            0|          467|          335|          15|          86|         102|         253|         384|          813|         4989|           0|           0|           0|           0|           0|            0|            0|           0|           2|           3|           9|          11|           21|          421|           4|           7|           7|          17|          26|           73|          201|           0.89|            0.0|           0.06|           0.05|          0.79|          0.91|          0.91|          0.91|          0.91|            0.9|           0.89|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|          0.02|          0.03|          0.03|          0.03|           0.02|           0.08|          0.21|          0.07|          0.06|          0.06|          0.06|           0.08|           0.04|          153|            0|           37|           75|               7|              22|              22|              30|              30|               31|               11|               0|               0|               0|               0|               0|                0|                0|               0|               2|               2|               5|               8|               12|                8|               3|               5|               6|              12|              14|               24|               11|     275.5|       0.0|      7.67|     22.33|       177.5|         0.0|         6.0|        12.0|   294.98|      0.0|     7.79|    26.14|      813|        0|       21|       73|              2575|                 0|               408|                41|                    2160|                       0|                      11|                     142|                0.85|                      0.93|                 0.0|                       0.0|                0.13|                       0.0|                0.01|                      0.06|           65|           73|            3|               1|
# |    6983|   1000|        598|    0|     1|  2| 708788|   300|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|        82733|          206|         4727|         4229|        3650|        9470|        4960|        4571|        7028|        13390|        39664|           8|           4|           3|           4|           3|           18|          166|         174|         285|         178|         118|         258|          301|         3413|         333|         452|         280|         206|         351|          958|         1649|            0.9|            0.0|           0.05|           0.05|          0.88|          0.93|          0.91|          0.93|          0.92|           0.91|           0.88|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.04|          0.03|          0.03|          0.02|          0.03|           0.02|           0.08|          0.08|          0.04|          0.05|          0.04|          0.05|           0.07|           0.04|          176|           42|          174|          185|              12|              30|              31|              31|              30|               31|               11|               6|               3|               3|               4|               3|               13|               10|              21|              29|              30|              25|              29|               31|                9|              21|              30|              31|              31|              30|               31|               11|           728|            92|           370|           462|           12|            5|            8|           10|            1|            0|            1|            1|            1|            1|            1|            1|   7178.17|      6.67|     219.0|     430.0|      5994.0|         4.0|       218.0|       342.0|  3688.56|     5.85|    72.81|   271.14|    13390|       18|      301|      958|              1.69|               48|             1.69|              1.0|              1.08|                3|             0.29|              1.0|              1.05|                6|             0.26|              1.0|              1.27|               12|             0.73|              1.0|                3.1|               124|              4.77|               1.0|               1.27|                 4|              0.64|               1.0|               1.13|                 8|              0.46|               1.0|               1.61|                25|              1.48|               1.0|             20505|                58|              3393|               310|                   16070|                      86|                      13|                    1064|                0.85|                      0.93|                 0.0|                       0.0|                0.14|                       0.0|                0.01|                      0.06|          528|          721|            3|               1|             3|             0|             1|             0|            0|            0|            0|            0|            0|             0|             3|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|            0.75|             0.0|            0.25|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.75|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.25|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             2|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 2|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              1|              0|              1|              0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                  1|                  0|                  1|                  0|                        2|                        0|                        0|                        0|                  0.5|                        1.0|                  0.0|                        0.0|                  0.5|                        0.0|                  0.0|                        0.0|            0|            0|         4727|            0|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            0|         174|         285|         178|         118|         258|          301|         3413|           0|           0|           0|           0|           0|            0|            0|            0.0|            0.0|            1.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           1.0|           1.0|           1.0|           1.0|           1.0|            1.0|            1.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            0|            0|          174|            0|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                0|              21|              29|              30|              25|              29|               31|                9|               0|               0|               0|               0|               0|                0|                0|       0.0|       0.0|     219.0|       0.0|         0.0|         0.0|       218.0|         0.0|      0.0|      0.0|    72.81|      0.0|        0|        0|      301|        0|                 0|                 0|              3393|                 0|                       0|                       0|                      13|                       0|                 0.0|                       0.0|                 0.0|                       0.0|                 1.0|                       1.0|                 0.0|                       0.0|          528|          721|            3|               1|
# |    8351|   1000|        598|    0|     1|  2| 708788|   300|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|        82733|          206|         4727|         4229|        3650|        9470|        4960|        4571|        7028|        13390|        39664|           8|           4|           3|           4|           3|           18|          166|         174|         285|         178|         118|         258|          301|         3413|         333|         452|         280|         206|         351|          958|         1649|            0.9|            0.0|           0.05|           0.05|          0.88|          0.93|          0.91|          0.93|          0.92|           0.91|           0.88|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.04|          0.03|          0.03|          0.02|          0.03|           0.02|           0.08|          0.08|          0.04|          0.05|          0.04|          0.05|           0.07|           0.04|          176|           42|          174|          185|              12|              30|              31|              31|              30|               31|               11|               6|               3|               3|               4|               3|               13|               10|              21|              29|              30|              25|              29|               31|                9|              21|              30|              31|              31|              30|               31|               11|           728|            92|           370|           462|           12|            5|            8|           10|            1|            0|            1|            1|            1|            1|            1|            1|   7178.17|      6.67|     219.0|     430.0|      5994.0|         4.0|       218.0|       342.0|  3688.56|     5.85|    72.81|   271.14|    13390|       18|      301|      958|              1.69|               48|             1.69|              1.0|              1.08|                3|             0.29|              1.0|              1.05|                6|             0.26|              1.0|              1.27|               12|             0.73|              1.0|                3.1|               124|              4.77|               1.0|               1.27|                 4|              0.64|               1.0|               1.13|                 8|              0.46|               1.0|               1.61|                25|              1.48|               1.0|             20505|                58|              3393|               310|                   16070|                      86|                      13|                    1064|                0.85|                      0.93|                 0.0|                       0.0|                0.14|                       0.0|                0.01|                      0.06|          528|          721|            3|               1|             3|             0|             1|             0|            0|            0|            0|            0|            0|             0|             3|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|            0.75|             0.0|            0.25|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.75|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.25|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             2|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 2|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              1|              0|              1|              0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                  1|                  0|                  1|                  0|                        2|                        0|                        0|                        0|                  0.5|                        1.0|                  0.0|                        0.0|                  0.5|                        0.0|                  0.0|                        0.0|        82733|            0|            0|         4229|        3650|        9470|        4960|        4571|        7028|        13390|        39664|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            0|         333|         452|         280|         206|         351|          958|         1649|           0.95|            0.0|            0.0|           0.05|          0.92|          0.95|          0.95|          0.96|          0.95|           0.93|           0.96|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.08|          0.05|          0.05|          0.04|          0.05|           0.07|           0.04|          176|            0|            0|          185|              12|              30|              31|              31|              30|               31|               11|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                0|              21|              30|              31|              31|              30|               31|               11|   7178.17|       0.0|       0.0|     430.0|      5994.0|         0.0|         0.0|       342.0|  3688.56|      0.0|      0.0|   271.14|    13390|        0|        0|      958|             20505|                 0|                 0|               310|                   16070|                       0|                       0|                    1064|                0.99|                      0.94|                 0.0|                       0.0|                 0.0|                       0.0|                0.01|                      0.06|         null|         null|         null|            null|
# |    6109| 100001|       1963|    0|     1|  0|1070887|  1023|           87|            0|            3|            0|           0|           0|           0|           1|           1|            1|           84|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            3|           0|           0|           0|           0|           0|            0|            0|           0.97|            0.0|           0.03|            0.0|           0.0|           0.0|           0.0|           1.0|           1.0|            1.0|           0.97|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.03|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            4|            0|            1|            0|               0|               0|               0|               1|               1|                1|                1|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                0|            74|             0|             3|             0|           20|            0|            3|            0|           35|            0|            3|            0|           40|            0|            3|            0|       0.5|       0.0|       0.0|       0.0|         0.5|         0.0|         0.0|         0.0|     0.55|      0.0|      0.0|      0.0|        1|        0|        0|        0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.85|                10|              1.97|               1.0|                0.0|                 0|               0.0|               0.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                84|                 0|                 3|                 0|                       0|                       0|                       0|                       0|                0.97|                       0.0|                 0.0|                       0.0|                0.03|                       0.0|                 0.0|                       0.0|        57321|           54|         1497|         1553|        2393|        5929|        1471|        1858|        2206|         5332|        38132|           0|           3|           1|           1|           1|            5|           43|          89|          81|          10|          16|          18|           13|         1270|         126|         204|          59|          51|          64|          309|          740|           0.95|            0.0|           0.02|           0.03|          0.92|          0.95|          0.95|          0.96|          0.96|           0.94|           0.95|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.03|          0.01|          0.01|          0.01|          0.01|            0.0|           0.03|          0.05|          0.03|          0.04|          0.03|          0.03|           0.05|           0.02|          176|           22|           77|          153|              12|              30|              31|              31|              30|               31|               11|               0|               3|               1|               1|               1|                5|               11|              18|              18|               8|               8|              13|               11|                1|              20|              28|              19|              21|              25|               29|               11|           500|            41|           190|           324|           28|            7|           17|           24|            1|            0|            1|            1|            1|            1|            1|            1|   3198.17|      1.83|     37.83|     135.5|      2299.5|         1.0|        17.0|        95.0|  1919.51|     1.83|    36.72|   102.95|     5929|        5|       89|      309|              1.54|               35|             1.42|              1.0|              1.11|                2|             0.32|              1.0|              1.02|                3|             0.13|              1.0|              1.27|               10|             0.75|              1.0|               3.75|               119|              6.05|               2.0|               1.23|                 4|              0.64|               1.0|               1.21|                 7|              0.57|               1.0|               1.81|                22|              1.75|               1.0|             20983|                 8|              1270|               102|                   13743|                      25|                       0|                     494|                0.94|                      0.96|                 0.0|                       0.0|                0.06|                       0.0|                 0.0|                      0.03|          205|          224|            3|               1|             6|             0|             1|             0|            0|            0|            0|            0|            0|             0|             6|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|            0.86|             0.0|            0.14|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.86|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.14|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             1|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              4|              0|              1|              0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                  6|                  0|                  1|                  0|                        0|                        0|                        0|                        0|                 0.86|                        0.0|                  0.0|                        0.0|                 0.14|                        0.0|                  0.0|                        0.0|        79322|            0|         2766|         2111|        3066|        7293|        2802|        2690|        3213|         6627|        53631|           0|           0|           0|           0|           0|            0|            0|         138|         123|          58|          61|          47|           44|         2295|         158|         281|         102|          81|          98|          373|         1018|           0.94|            0.0|           0.03|           0.03|          0.91|          0.95|          0.95|          0.95|          0.96|           0.94|           0.94|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.04|          0.02|          0.02|          0.02|          0.01|           0.01|           0.04|          0.05|          0.04|          0.03|          0.03|          0.03|           0.05|           0.02|          176|            0|          141|          170|              12|              30|              31|              31|              30|               31|               11|               0|               0|               0|               0|               0|                0|                0|              19|              24|              21|              21|              24|               25|                7|              20|              29|              28|              25|              27|               30|               11|   4281.83|       0.0|      78.5|    182.17|      3139.5|         0.0|        59.5|       130.0|  2093.39|      0.0|    41.06|   118.76|     7293|        0|      138|      373|             31653|                 0|              2280|               173|                   18219|                       0|                      14|                     676|                0.93|                      0.96|                 0.0|                       0.0|                0.07|                       0.0|                0.01|                      0.04|          445|          503|            3|               1|
# +--------+-------+-----------+-----+------+---+-------+------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+--------------+--------------+--------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+----------+----------+----------+----------+-------------------+-------------------+-------------------+-------------------+-------------------------+-------------------------+-------------------------+-------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+


In [ ]:
test_feature_5 = test_feature_4 \
                .join(cat_feature, "cat_id", "left") \
                .orderBy("user_id", "merchant_id")

# +------+--------+-------+-----------+-----+------+---+-------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+--------------+--------------+--------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+----------+----------+----------+----------+-------------------+-------------------+-------------------+-------------------+-------------------------+-------------------------+-------------------------+-------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+----------------+----------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+
# |cat_id|brand_id|user_id|merchant_id|label|gender|age|item_id|u_ttl_clk_cnt|u_ttl_atc_cnt|u_ttl_buy_cnt|u_ttl_atf_cnt|u_clk_cnt_m5|u_clk_cnt_m6|u_clk_cnt_m7|u_clk_cnt_m8|u_clk_cnt_m9|u_clk_cnt_m10|u_clk_cnt_m11|u_atc_cnt_m5|u_atc_cnt_m6|u_atc_cnt_m7|u_atc_cnt_m8|u_atc_cnt_m9|u_atc_cnt_m10|u_atc_cnt_m11|u_buy_cnt_m5|u_buy_cnt_m6|u_buy_cnt_m7|u_buy_cnt_m8|u_buy_cnt_m9|u_buy_cnt_m10|u_buy_cnt_m11|u_atf_cnt_m5|u_atf_cnt_m6|u_atf_cnt_m7|u_atf_cnt_m8|u_atf_cnt_m9|u_atf_cnt_m10|u_atf_cnt_m11|u_ttl_clk_ratio|u_ttl_atc_ratio|u_ttl_buy_ratio|u_ttl_atf_ratio|u_clk_ratio_m5|u_clk_ratio_m6|u_clk_ratio_m7|u_clk_ratio_m8|u_clk_ratio_m9|u_clk_ratio_m10|u_clk_ratio_m11|u_atc_ratio_m5|u_atc_ratio_m6|u_atc_ratio_m7|u_atc_ratio_m8|u_atc_ratio_m9|u_atc_ratio_m10|u_atc_ratio_m11|u_buy_ratio_m5|u_buy_ratio_m6|u_buy_ratio_m7|u_buy_ratio_m8|u_buy_ratio_m9|u_buy_ratio_m10|u_buy_ratio_m11|u_atf_ratio_m5|u_atf_ratio_m6|u_atf_ratio_m7|u_atf_ratio_m8|u_atf_ratio_m9|u_atf_ratio_m10|u_atf_ratio_m11|u_clk_day_cnt|u_atc_day_cnt|u_buy_day_cnt|u_atf_day_cnt|u_clk_day_cnt_m5|u_clk_day_cnt_m6|u_clk_day_cnt_m7|u_clk_day_cnt_m8|u_clk_day_cnt_m9|u_clk_day_cnt_m10|u_clk_day_cnt_m11|u_atc_day_cnt_m5|u_atc_day_cnt_m6|u_atc_day_cnt_m7|u_atc_day_cnt_m8|u_atc_day_cnt_m9|u_atc_day_cnt_m10|u_atc_day_cnt_m11|u_buy_day_cnt_m5|u_buy_day_cnt_m6|u_buy_day_cnt_m7|u_buy_day_cnt_m8|u_buy_day_cnt_m9|u_buy_day_cnt_m10|u_buy_day_cnt_m11|u_atf_day_cnt_m5|u_atf_day_cnt_m6|u_atf_day_cnt_m7|u_atf_day_cnt_m8|u_atf_day_cnt_m9|u_atf_day_cnt_m10|u_atf_day_cnt_m11|u_clk_item_div|u_atc_item_div|u_buy_item_div|u_atf_item_div|u_clk_cat_div|u_atc_cat_div|u_buy_cat_div|u_atf_cat_div|u_clk_brd_div|u_atc_brd_div|u_buy_brd_div|u_atf_brd_div|u_clk_mer_div|u_atc_mer_div|u_buy_mer_div|u_atf_mer_div|u_clk_mean|u_atc_mean|u_buy_mean|u_atf_mean|u_clk_median|u_atc_median|u_buy_median|u_atf_median|u_clk_std|u_atc_std|u_buy_std|u_atf_std|u_clk_max|u_atc_max|u_buy_max|u_atf_max|u_clk_day_cnt_mean|u_clk_day_cnt_max|u_clk_day_cnt_std|u_clk_day_cnt_med|u_atc_day_cnt_mean|u_atc_day_cnt_max|u_atc_day_cnt_std|u_atc_day_cnt_med|u_buy_day_cnt_mean|u_buy_day_cnt_max|u_buy_day_cnt_std|u_buy_day_cnt_med|u_atf_day_cnt_mean|u_atf_day_cnt_max|u_atf_day_cnt_std|u_atf_day_cnt_med|u_clk_item_cnt_mean|u_clk_item_cnt_max|u_clk_item_cnt_std|u_clk_item_cnt_med|u_atc_item_cnt_mean|u_atc_item_cnt_max|u_atc_item_cnt_std|u_atc_item_cnt_med|u_buy_item_cnt_mean|u_buy_item_cnt_max|u_buy_item_cnt_std|u_buy_item_cnt_med|u_atf_item_cnt_mean|u_atf_item_cnt_max|u_atf_item_cnt_std|u_atf_item_cnt_med|u_clk_cnt_double11|u_atc_cnt_double11|u_buy_cnt_double11|u_atf_cnt_double11|u_clk_cnt_1wpre_double11|u_atc_cnt_1wpre_double11|u_buy_cnt_1wpre_double11|u_atf_cnt_1wpre_double11|u_clk_ratio_double11|u_clk_ratio_1wpre_double11|u_atc_ratio_double11|u_atc_ratio_1wpre_double11|u_buy_ratio_double11|u_buy_ratio_1wpre_double11|u_atf_ratio_double11|u_atf_ratio_1wpre_double11|m_ttl_clk_cnt|m_ttl_atc_cnt|m_ttl_buy_cnt|m_ttl_atf_cnt|m_clk_cnt_m5|m_clk_cnt_m6|m_clk_cnt_m7|m_clk_cnt_m8|m_clk_cnt_m9|m_clk_cnt_m10|m_clk_cnt_m11|m_atc_cnt_m5|m_atc_cnt_m6|m_atc_cnt_m7|m_atc_cnt_m8|m_atc_cnt_m9|m_atc_cnt_m10|m_atc_cnt_m11|m_buy_cnt_m5|m_buy_cnt_m6|m_buy_cnt_m7|m_buy_cnt_m8|m_buy_cnt_m9|m_buy_cnt_m10|m_buy_cnt_m11|m_atf_cnt_m5|m_atf_cnt_m6|m_atf_cnt_m7|m_atf_cnt_m8|m_atf_cnt_m9|m_atf_cnt_m10|m_atf_cnt_m11|m_ttl_clk_ratio|m_ttl_atc_ratio|m_ttl_buy_ratio|m_ttl_atf_ratio|m_clk_ratio_m5|m_clk_ratio_m6|m_clk_ratio_m7|m_clk_ratio_m8|m_clk_ratio_m9|m_clk_ratio_m10|m_clk_ratio_m11|m_atc_ratio_m5|m_atc_ratio_m6|m_atc_ratio_m7|m_atc_ratio_m8|m_atc_ratio_m9|m_atc_ratio_m10|m_atc_ratio_m11|m_buy_ratio_m5|m_buy_ratio_m6|m_buy_ratio_m7|m_buy_ratio_m8|m_buy_ratio_m9|m_buy_ratio_m10|m_buy_ratio_m11|m_atf_ratio_m5|m_atf_ratio_m6|m_atf_ratio_m7|m_atf_ratio_m8|m_atf_ratio_m9|m_atf_ratio_m10|m_atf_ratio_m11|m_clk_day_cnt|m_atc_day_cnt|m_buy_day_cnt|m_atf_day_cnt|m_clk_day_cnt_m5|m_clk_day_cnt_m6|m_clk_day_cnt_m7|m_clk_day_cnt_m8|m_clk_day_cnt_m9|m_clk_day_cnt_m10|m_clk_day_cnt_m11|m_atc_day_cnt_m5|m_atc_day_cnt_m6|m_atc_day_cnt_m7|m_atc_day_cnt_m8|m_atc_day_cnt_m9|m_atc_day_cnt_m10|m_atc_day_cnt_m11|m_buy_day_cnt_m5|m_buy_day_cnt_m6|m_buy_day_cnt_m7|m_buy_day_cnt_m8|m_buy_day_cnt_m9|m_buy_day_cnt_m10|m_buy_day_cnt_m11|m_atf_day_cnt_m5|m_atf_day_cnt_m6|m_atf_day_cnt_m7|m_atf_day_cnt_m8|m_atf_day_cnt_m9|m_atf_day_cnt_m10|m_atf_day_cnt_m11|m_clk_item_div|m_atc_item_div|m_buy_item_div|m_atf_item_div|m_clk_cat_div|m_atc_cat_div|m_buy_cat_div|m_atf_cat_div|m_clk_brd_div|m_atc_brd_div|m_buy_brd_div|m_atf_brd_div|m_clk_mer_div|m_atc_mer_div|m_buy_mer_div|m_atf_mer_div|m_clk_mean|m_atc_mean|m_buy_mean|m_atf_mean|m_clk_median|m_atc_median|m_buy_median|m_atf_median|m_clk_std|m_atc_std|m_buy_std|m_atf_std|m_clk_max|m_atc_max|m_buy_max|m_atf_max|m_clk_day_cnt_mean|m_clk_day_cnt_max|m_clk_day_cnt_std|m_clk_day_cnt_med|m_atc_day_cnt_mean|m_atc_day_cnt_max|m_atc_day_cnt_std|m_atc_day_cnt_med|m_buy_day_cnt_mean|m_buy_day_cnt_max|m_buy_day_cnt_std|m_buy_day_cnt_med|m_atf_day_cnt_mean|m_atf_day_cnt_max|m_atf_day_cnt_std|m_atf_day_cnt_med|m_clk_item_cnt_mean|m_clk_item_cnt_max|m_clk_item_cnt_std|m_clk_item_cnt_med|m_atc_item_cnt_mean|m_atc_item_cnt_max|m_atc_item_cnt_std|m_atc_item_cnt_med|m_buy_item_cnt_mean|m_buy_item_cnt_max|m_buy_item_cnt_std|m_buy_item_cnt_med|m_atf_item_cnt_mean|m_atf_item_cnt_max|m_atf_item_cnt_std|m_atf_item_cnt_med|m_clk_cnt_double11|m_atc_cnt_double11|m_buy_cnt_double11|m_atf_cnt_double11|m_clk_cnt_1wpre_double11|m_atc_cnt_1wpre_double11|m_buy_cnt_1wpre_double11|m_atf_cnt_1wpre_double11|m_clk_ratio_double11|m_clk_ratio_1wpre_double11|m_atc_ratio_double11|m_atc_ratio_1wpre_double11|m_buy_ratio_double11|m_buy_ratio_1wpre_double11|m_atf_ratio_double11|m_atf_ratio_1wpre_double11|m_rpt_byr_num|m_rpt_byr_day|m_popular_age|m_popular_gender|um_ttl_clk_cnt|um_ttl_atc_cnt|um_ttl_buy_cnt|um_ttl_atf_cnt|um_clk_cnt_m5|um_clk_cnt_m6|um_clk_cnt_m7|um_clk_cnt_m8|um_clk_cnt_m9|um_clk_cnt_m10|um_clk_cnt_m11|um_atc_cnt_m5|um_atc_cnt_m6|um_atc_cnt_m7|um_atc_cnt_m8|um_atc_cnt_m9|um_atc_cnt_m10|um_atc_cnt_m11|um_buy_cnt_m5|um_buy_cnt_m6|um_buy_cnt_m7|um_buy_cnt_m8|um_buy_cnt_m9|um_buy_cnt_m10|um_buy_cnt_m11|um_atf_cnt_m5|um_atf_cnt_m6|um_atf_cnt_m7|um_atf_cnt_m8|um_atf_cnt_m9|um_atf_cnt_m10|um_atf_cnt_m11|um_ttl_clk_ratio|um_ttl_atc_ratio|um_ttl_buy_ratio|um_ttl_atf_ratio|um_clk_ratio_m5|um_clk_ratio_m6|um_clk_ratio_m7|um_clk_ratio_m8|um_clk_ratio_m9|um_clk_ratio_m10|um_clk_ratio_m11|um_atc_ratio_m5|um_atc_ratio_m6|um_atc_ratio_m7|um_atc_ratio_m8|um_atc_ratio_m9|um_atc_ratio_m10|um_atc_ratio_m11|um_buy_ratio_m5|um_buy_ratio_m6|um_buy_ratio_m7|um_buy_ratio_m8|um_buy_ratio_m9|um_buy_ratio_m10|um_buy_ratio_m11|um_atf_ratio_m5|um_atf_ratio_m6|um_atf_ratio_m7|um_atf_ratio_m8|um_atf_ratio_m9|um_atf_ratio_m10|um_atf_ratio_m11|um_clk_day_cnt|um_atc_day_cnt|um_buy_day_cnt|um_atf_day_cnt|um_clk_day_cnt_m5|um_clk_day_cnt_m6|um_clk_day_cnt_m7|um_clk_day_cnt_m8|um_clk_day_cnt_m9|um_clk_day_cnt_m10|um_clk_day_cnt_m11|um_atc_day_cnt_m5|um_atc_day_cnt_m6|um_atc_day_cnt_m7|um_atc_day_cnt_m8|um_atc_day_cnt_m9|um_atc_day_cnt_m10|um_atc_day_cnt_m11|um_buy_day_cnt_m5|um_buy_day_cnt_m6|um_buy_day_cnt_m7|um_buy_day_cnt_m8|um_buy_day_cnt_m9|um_buy_day_cnt_m10|um_buy_day_cnt_m11|um_atf_day_cnt_m5|um_atf_day_cnt_m6|um_atf_day_cnt_m7|um_atf_day_cnt_m8|um_atf_day_cnt_m9|um_atf_day_cnt_m10|um_atf_day_cnt_m11|um_clk_item_div|um_atc_item_div|um_buy_item_div|um_atf_item_div|um_clk_cat_div|um_atc_cat_div|um_buy_cat_div|um_atf_cat_div|um_clk_brd_div|um_atc_brd_div|um_buy_brd_div|um_atf_brd_div|um_clk_mer_div|um_atc_mer_div|um_buy_mer_div|um_atf_mer_div|um_clk_mean|um_atc_mean|um_buy_mean|um_atf_mean|um_clk_median|um_atc_median|um_buy_median|um_atf_median|um_clk_std|um_atc_std|um_buy_std|um_atf_std|um_clk_max|um_atc_max|um_buy_max|um_atf_max|um_clk_cnt_double11|um_atc_cnt_double11|um_buy_cnt_double11|um_atf_cnt_double11|um_clk_cnt_1wpre_double11|um_atc_cnt_1wpre_double11|um_buy_cnt_1wpre_double11|um_atf_cnt_1wpre_double11|um_clk_ratio_double11|um_clk_ratio_1wpre_double11|um_atc_ratio_double11|um_atc_ratio_1wpre_double11|um_buy_ratio_double11|um_buy_ratio_1wpre_double11|um_atf_ratio_double11|um_atf_ratio_1wpre_double11|b_ttl_clk_cnt|b_ttl_atc_cnt|b_ttl_buy_cnt|b_ttl_atf_cnt|b_clk_cnt_m5|b_clk_cnt_m6|b_clk_cnt_m7|b_clk_cnt_m8|b_clk_cnt_m9|b_clk_cnt_m10|b_clk_cnt_m11|b_atc_cnt_m5|b_atc_cnt_m6|b_atc_cnt_m7|b_atc_cnt_m8|b_atc_cnt_m9|b_atc_cnt_m10|b_atc_cnt_m11|b_buy_cnt_m5|b_buy_cnt_m6|b_buy_cnt_m7|b_buy_cnt_m8|b_buy_cnt_m9|b_buy_cnt_m10|b_buy_cnt_m11|b_atf_cnt_m5|b_atf_cnt_m6|b_atf_cnt_m7|b_atf_cnt_m8|b_atf_cnt_m9|b_atf_cnt_m10|b_atf_cnt_m11|b_ttl_clk_ratio|b_ttl_atc_ratio|b_ttl_buy_ratio|b_ttl_atf_ratio|b_clk_ratio_m5|b_clk_ratio_m6|b_clk_ratio_m7|b_clk_ratio_m8|b_clk_ratio_m9|b_clk_ratio_m10|b_clk_ratio_m11|b_atc_ratio_m5|b_atc_ratio_m6|b_atc_ratio_m7|b_atc_ratio_m8|b_atc_ratio_m9|b_atc_ratio_m10|b_atc_ratio_m11|b_buy_ratio_m5|b_buy_ratio_m6|b_buy_ratio_m7|b_buy_ratio_m8|b_buy_ratio_m9|b_buy_ratio_m10|b_buy_ratio_m11|b_atf_ratio_m5|b_atf_ratio_m6|b_atf_ratio_m7|b_atf_ratio_m8|b_atf_ratio_m9|b_atf_ratio_m10|b_atf_ratio_m11|b_clk_day_cnt|b_atc_day_cnt|b_buy_day_cnt|b_atf_day_cnt|b_clk_day_cnt_m5|b_clk_day_cnt_m6|b_clk_day_cnt_m7|b_clk_day_cnt_m8|b_clk_day_cnt_m9|b_clk_day_cnt_m10|b_clk_day_cnt_m11|b_atc_day_cnt_m5|b_atc_day_cnt_m6|b_atc_day_cnt_m7|b_atc_day_cnt_m8|b_atc_day_cnt_m9|b_atc_day_cnt_m10|b_atc_day_cnt_m11|b_buy_day_cnt_m5|b_buy_day_cnt_m6|b_buy_day_cnt_m7|b_buy_day_cnt_m8|b_buy_day_cnt_m9|b_buy_day_cnt_m10|b_buy_day_cnt_m11|b_atf_day_cnt_m5|b_atf_day_cnt_m6|b_atf_day_cnt_m7|b_atf_day_cnt_m8|b_atf_day_cnt_m9|b_atf_day_cnt_m10|b_atf_day_cnt_m11|b_clk_mean|b_atc_mean|b_buy_mean|b_atf_mean|b_clk_median|b_atc_median|b_buy_median|b_atf_median|b_clk_std|b_atc_std|b_buy_std|b_atf_std|b_clk_max|b_atc_max|b_buy_max|b_atf_max|b_clk_cnt_double11|b_atc_cnt_double11|b_buy_cnt_double11|b_atf_cnt_double11|b_clk_cnt_1wpre_double11|b_atc_cnt_1wpre_double11|b_buy_cnt_1wpre_double11|b_atf_cnt_1wpre_double11|b_clk_ratio_double11|b_clk_ratio_1wpre_double11|b_atc_ratio_double11|b_atc_ratio_1wpre_double11|b_buy_ratio_double11|b_buy_ratio_1wpre_double11|b_atf_ratio_double11|b_atf_ratio_1wpre_double11|b_rpt_byr_num|b_rpt_byr_day|b_popular_age|b_popular_gender|c_ttl_clk_cnt|c_ttl_atc_cnt|c_ttl_buy_cnt|c_ttl_atf_cnt|c_clk_cnt_m5|c_clk_cnt_m6|c_clk_cnt_m7|c_clk_cnt_m8|c_clk_cnt_m9|c_clk_cnt_m10|c_clk_cnt_m11|c_atc_cnt_m5|c_atc_cnt_m6|c_atc_cnt_m7|c_atc_cnt_m8|c_atc_cnt_m9|c_atc_cnt_m10|c_atc_cnt_m11|c_buy_cnt_m5|c_buy_cnt_m6|c_buy_cnt_m7|c_buy_cnt_m8|c_buy_cnt_m9|c_buy_cnt_m10|c_buy_cnt_m11|c_atf_cnt_m5|c_atf_cnt_m6|c_atf_cnt_m7|c_atf_cnt_m8|c_atf_cnt_m9|c_atf_cnt_m10|c_atf_cnt_m11|c_ttl_clk_ratio|c_ttl_atc_ratio|c_ttl_buy_ratio|c_ttl_atf_ratio|c_clk_ratio_m5|c_clk_ratio_m6|c_clk_ratio_m7|c_clk_ratio_m8|c_clk_ratio_m9|c_clk_ratio_m10|c_clk_ratio_m11|c_atc_ratio_m5|c_atc_ratio_m6|c_atc_ratio_m7|c_atc_ratio_m8|c_atc_ratio_m9|c_atc_ratio_m10|c_atc_ratio_m11|c_buy_ratio_m5|c_buy_ratio_m6|c_buy_ratio_m7|c_buy_ratio_m8|c_buy_ratio_m9|c_buy_ratio_m10|c_buy_ratio_m11|c_atf_ratio_m5|c_atf_ratio_m6|c_atf_ratio_m7|c_atf_ratio_m8|c_atf_ratio_m9|c_atf_ratio_m10|c_atf_ratio_m11|c_clk_day_cnt|c_atc_day_cnt|c_buy_day_cnt|c_atf_day_cnt|c_clk_day_cnt_m5|c_clk_day_cnt_m6|c_clk_day_cnt_m7|c_clk_day_cnt_m8|c_clk_day_cnt_m9|c_clk_day_cnt_m10|c_clk_day_cnt_m11|c_atc_day_cnt_m5|c_atc_day_cnt_m6|c_atc_day_cnt_m7|c_atc_day_cnt_m8|c_atc_day_cnt_m9|c_atc_day_cnt_m10|c_atc_day_cnt_m11|c_buy_day_cnt_m5|c_buy_day_cnt_m6|c_buy_day_cnt_m7|c_buy_day_cnt_m8|c_buy_day_cnt_m9|c_buy_day_cnt_m10|c_buy_day_cnt_m11|c_atf_day_cnt_m5|c_atf_day_cnt_m6|c_atf_day_cnt_m7|c_atf_day_cnt_m8|c_atf_day_cnt_m9|c_atf_day_cnt_m10|c_atf_day_cnt_m11|c_clk_mean|c_atc_mean|c_buy_mean|c_atf_mean|c_clk_median|c_atc_median|c_buy_median|c_atf_median|c_clk_std|c_atc_std|c_buy_std|c_atf_std|c_clk_max|c_atc_max|c_buy_max|c_atf_max|c_clk_cnt_double11|c_atc_cnt_double11|c_buy_cnt_double11|c_atf_cnt_double11|c_clk_cnt_1wpre_double11|c_atc_cnt_1wpre_double11|c_buy_cnt_1wpre_double11|c_atf_cnt_1wpre_double11|ttl_cnt_double11|ttl_cnt_1wpre_double11|c_clk_ratio_double11|c_clk_ratio_1wpre_double11|c_atc_ratio_double11|c_atc_ratio_1wpre_double11|c_buy_ratio_double11|c_buy_ratio_1wpre_double11|c_atf_ratio_double11|c_atf_ratio_1wpre_double11|c_rpt_byr_num|c_rpt_byr_day|c_popular_age|c_popular_gender|
# +------+--------+-------+-----------+-----+------+---+-------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+--------------+--------------+--------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+----------+----------+----------+----------+-------------------+-------------------+-------------------+-------------------+-------------------------+-------------------------+-------------------------+-------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+----------------+----------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+
# |   992|    6805|      1|       1019|    1|     1|  3|1110495|           27|            0|            6|            0|           0|           0|           0|           0|           0|           14|           13|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            2|            4|           0|           0|           0|           0|           0|            0|            0|           0.82|            0.0|           0.18|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|           0.88|           0.76|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|           0.13|           0.24|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            5|            0|            3|            0|               0|               0|               0|               0|               0|                4|                1|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                2|                1|               0|               0|               0|               0|               0|                0|                0|            12|             0|             3|             0|            6|            0|            3|            0|            9|            0|            3|            0|            9|            0|            3|            0|      2.33|       0.0|      0.33|       0.0|         0.0|         0.0|         0.0|         0.0|     5.72|      0.0|     0.82|      0.0|       14|        0|        2|        0|              1.11|                2|             0.33|              1.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.33|                 4|               1.0|               1.0|                0.0|                 0|               0.0|               0.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                13|                 0|                 4|                 0|                       0|                       0|                       0|                       0|                0.76|                       0.0|                 0.0|                       0.0|                0.24|                       0.0|                 0.0|                       0.0|         6003|            5|          941|          484|         463|         678|         459|         511|         727|         1263|         1902|           0|           0|           1|           1|           0|            2|            1|         182|          83|          38|          67|          96|          180|          295|          85|          53|          32|          35|          48|          127|          104|           0.81|            0.0|           0.13|           0.07|          0.63|          0.83|          0.87|          0.83|          0.83|            0.8|           0.83|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.25|           0.1|          0.07|          0.11|          0.11|           0.11|           0.13|          0.12|          0.07|          0.06|          0.06|          0.06|           0.08|           0.05|          174|            5|          139|          142|              12|              30|              29|              31|              30|               31|               11|               0|               0|               1|               1|               0|                2|                1|              20|              20|              15|              20|              24|               29|               11|              21|              23|              17|              20|              20|               31|               10|           147|             3|            61|            60|            8|            2|            8|            8|            1|            0|            1|            1|            1|            1|            1|            1|     683.5|      0.67|    107.67|     63.33|       594.5|         0.5|        89.5|        50.5|   305.46|     0.82|    60.02|    36.47|     1263|        2|      182|      127|              1.17|               10|             0.57|              1.0|               1.0|                1|              0.0|              1.0|              1.05|                3|             0.23|              1.0|              1.04|                3|             0.21|              1.0|               1.33|                14|              0.99|               1.0|                1.0|                 1|               0.0|               1.0|               1.16|                 4|              0.45|               1.0|               1.07|                 6|              0.37|               1.0|              1167|                 1|               234|                37|                     544|                       0|                      40|                      51|                0.81|                      0.86|                 0.0|                       0.0|                0.16|                      0.06|                0.03|                      0.08|          191|          215|            3|               0|            10|             0|             4|             0|            0|            0|            0|            0|            0|             0|            10|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             4|            0|            0|            0|            0|            0|             0|             0|            0.71|             0.0|            0.29|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.71|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.29|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             1|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              1|              0|              1|              0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                 10|                  0|                  4|                  0|                        0|                        0|                        0|                        0|                 0.71|                        0.0|                  0.0|                        0.0|                 0.29|                        0.0|                  0.0|                        0.0|         6003|            0|          941|          484|         463|         678|         459|         511|         727|         1263|         1902|           0|           0|           0|           0|           0|            0|            0|         182|          83|          38|          67|          96|          180|          295|          85|          53|          32|          35|          48|          127|          104|           0.81|            0.0|           0.13|           0.07|          0.63|          0.83|          0.87|          0.83|          0.83|            0.8|           0.83|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.25|           0.1|          0.07|          0.11|          0.11|           0.11|           0.13|          0.12|          0.07|          0.06|          0.06|          0.06|           0.08|           0.05|          174|            0|          139|          142|              12|              30|              29|              31|              30|               31|               11|               0|               0|               0|               0|               0|                0|                0|              20|              20|              15|              20|              24|               29|               11|              21|              23|              17|              20|              20|               31|               10|     683.5|       0.0|    107.67|     63.33|       594.5|         0.0|        89.5|        50.5|   305.46|      0.0|    60.02|    36.47|     1263|        0|      182|      127|              1167|                 0|               234|                37|                     544|                       0|                      40|                      51|                0.81|                      0.86|                 0.0|                       0.0|                0.16|                      0.06|                0.03|                      0.08|          191|          215|            3|               0|       233541|          581|        25044|        15793|        1956|        5204|        5302|       11545|       27907|        66085|       115542|           2|           4|           5|           7|          41|          128|          394|         271|         382|         442|        1012|        2658|         7120|        13159|         270|         348|         403|         890|        2278|         5810|         5794|           0.85|            0.0|           0.09|           0.06|          0.78|          0.88|          0.86|          0.86|          0.85|           0.84|           0.86|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.11|          0.06|          0.07|          0.08|          0.08|           0.09|            0.1|          0.11|          0.06|          0.07|          0.07|          0.07|           0.07|           0.04|          176|           75|          185|          185|              12|              30|              31|              31|              30|               31|               11|               2|               3|               5|               5|              22|               27|               11|              21|              30|              31|              31|              30|               31|               11|              21|              30|              31|              31|              30|               31|               11|   19666.5|     31.17|   1980.83|    1666.5|      8423.5|         6.0|       727.0|       646.5| 24567.08|    49.66|  2670.43|  2164.74|    66085|      128|     7120|     5810|             58739|               122|             10939|              1157|                   46044|                     242|                    1578|                    3679|           70957|                 51543|                0.83|                      0.89|                 0.0|                       0.0|                0.15|                      0.03|                0.02|                      0.07|         4780|         6148|            3|               0|
# |     2|    8055|   1000|       3819|    0|     1|  2| 517962|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|         6642|           23|          467|          335|          15|          86|         102|         253|         384|          813|         4989|           0|           0|           0|           0|           1|            1|           21|           0|           2|           3|           9|          11|           21|          421|           4|           7|           7|          17|          26|           73|          201|           0.89|            0.0|           0.06|           0.04|          0.79|          0.91|          0.91|          0.91|          0.91|            0.9|           0.89|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|          0.02|          0.03|          0.03|          0.03|           0.02|           0.07|          0.21|          0.07|          0.06|          0.06|          0.06|           0.08|           0.04|          153|            8|           37|           75|               7|              22|              22|              30|              30|               31|               11|               0|               0|               0|               0|               1|                1|                6|               0|               2|               2|               5|               8|               12|                8|               3|               5|               6|              12|              14|               24|               11|            95|            12|            40|            38|           14|            6|           12|           11|            1|            0|            1|            1|            1|            1|            1|            1|     275.5|      0.33|      7.67|     22.33|       177.5|         0.0|         6.0|        12.0|   294.98|     0.52|     7.79|    26.14|      813|        1|       21|       73|              1.26|               13|             0.76|              1.0|              1.05|                2|             0.22|              1.0|              1.02|                3|             0.16|              1.0|               1.1|                3|             0.32|              1.0|               1.88|                29|              2.09|               1.0|                1.1|                 2|              0.31|               1.0|               1.21|                 4|              0.53|               1.0|               1.25|                 5|              0.63|               1.0|              2575|                 7|               408|                41|                    2160|                      12|                      11|                     142|                0.85|                      0.93|                 0.0|                      0.01|                0.13|                       0.0|                0.01|                      0.06|           65|           73|            3|               1|             5|             0|             1|             0|            0|            0|            0|            0|            0|             0|             5|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|            0.83|             0.0|            0.17|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.83|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.17|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             2|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 2|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              2|              0|              1|              0|             2|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                  1|                  0|                  1|                  0|                        4|                        0|                        0|                        0|                  0.5|                        1.0|                  0.0|                        0.0|                  0.5|                        0.0|                  0.0|                        0.0|         6642|            0|          467|          335|          15|          86|         102|         253|         384|          813|         4989|           0|           0|           0|           0|           0|            0|            0|           0|           2|           3|           9|          11|           21|          421|           4|           7|           7|          17|          26|           73|          201|           0.89|            0.0|           0.06|           0.05|          0.79|          0.91|          0.91|          0.91|          0.91|            0.9|           0.89|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|          0.02|          0.03|          0.03|          0.03|           0.02|           0.08|          0.21|          0.07|          0.06|          0.06|          0.06|           0.08|           0.04|          153|            0|           37|           75|               7|              22|              22|              30|              30|               31|               11|               0|               0|               0|               0|               0|                0|                0|               0|               2|               2|               5|               8|               12|                8|               3|               5|               6|              12|              14|               24|               11|     275.5|       0.0|      7.67|     22.33|       177.5|         0.0|         6.0|        12.0|   294.98|      0.0|     7.79|    26.14|      813|        0|       21|       73|              2575|                 0|               408|                41|                    2160|                       0|                      11|                     142|                0.85|                      0.93|                 0.0|                       0.0|                0.13|                       0.0|                0.01|                      0.06|           65|           73|            3|               1|       113178|          313|         4442|         5907|          52|         269|         338|        1734|        3869|        19450|        87466|           0|           0|           0|           1|           4|           23|          285|           1|           1|           4|          58|          84|          473|         3821|          10|          34|          43|         121|         316|         1774|         3609|           0.91|            0.0|           0.04|           0.05|          0.83|          0.88|          0.88|          0.91|          0.91|            0.9|           0.92|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.02|           0.0|          0.01|          0.03|          0.02|           0.02|           0.04|          0.16|          0.11|          0.11|          0.06|          0.07|           0.08|           0.04|          175|           30|           92|          143|              12|              29|              31|              31|              30|               31|               11|               0|               0|               0|               1|               4|               14|               11|               1|               1|               4|              16|              28|               31|               11|               8|              21|              15|              27|              30|               31|               11|   4285.33|      4.67|     103.5|     383.0|      1036.0|         0.5|        31.0|        82.0|  7566.81|     9.11|   184.33|   690.59|    19450|       23|      473|     1774|             42702|                94|              3639|               749|                   37529|                     164|                     153|                    2352|           47184|                 40198|                0.91|                      0.93|                 0.0|                       0.0|                0.08|                       0.0|                0.02|                      0.06|          220|          271|            3|               0|
# |   300|    6983|   1000|        598|    0|     1|  2| 708788|           66|            0|            7|            6|           0|           0|           0|           0|           0|            0|           66|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            7|           0|           0|           0|           0|           0|            0|            6|           0.84|            0.0|           0.09|           0.08|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.09|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.08|            2|            0|            1|            1|               0|               0|               0|               0|               0|                0|                2|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                1|            21|             0|             7|             6|            5|            0|            4|            4|            9|            0|            5|            3|            9|            0|            5|            3|       0.0|       0.0|       0.0|       0.0|         0.0|         0.0|         0.0|         0.0|      0.0|      0.0|      0.0|      0.0|        0|        0|        0|        0|              1.56|                2|             0.53|              2.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               1.0|                1|              0.0|              1.0|               2.33|                11|              3.28|               1.0|                0.0|                 0|               0.0|               0.0|                1.4|                 3|              0.89|               1.0|                2.0|                 4|              1.73|               1.0|                18|                 0|                 7|                 0|                      48|                       0|                       0|                       6|                0.72|                      0.89|                 0.0|                       0.0|                0.28|                       0.0|                 0.0|                      0.11|        82733|          206|         4727|         4229|        3650|        9470|        4960|        4571|        7028|        13390|        39664|           8|           4|           3|           4|           3|           18|          166|         174|         285|         178|         118|         258|          301|         3413|         333|         452|         280|         206|         351|          958|         1649|            0.9|            0.0|           0.05|           0.05|          0.88|          0.93|          0.91|          0.93|          0.92|           0.91|           0.88|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.04|          0.03|          0.03|          0.02|          0.03|           0.02|           0.08|          0.08|          0.04|          0.05|          0.04|          0.05|           0.07|           0.04|          176|           42|          174|          185|              12|              30|              31|              31|              30|               31|               11|               6|               3|               3|               4|               3|               13|               10|              21|              29|              30|              25|              29|               31|                9|              21|              30|              31|              31|              30|               31|               11|           728|            92|           370|           462|           12|            5|            8|           10|            1|            0|            1|            1|            1|            1|            1|            1|   7178.17|      6.67|     219.0|     430.0|      5994.0|         4.0|       218.0|       342.0|  3688.56|     5.85|    72.81|   271.14|    13390|       18|      301|      958|              1.69|               48|             1.69|              1.0|              1.08|                3|             0.29|              1.0|              1.05|                6|             0.26|              1.0|              1.27|               12|             0.73|              1.0|                3.1|               124|              4.77|               1.0|               1.27|                 4|              0.64|               1.0|               1.13|                 8|              0.46|               1.0|               1.61|                25|              1.48|               1.0|             20505|                58|              3393|               310|                   16070|                      86|                      13|                    1064|                0.85|                      0.93|                 0.0|                       0.0|                0.14|                       0.0|                0.01|                      0.06|          528|          721|            3|               1|             3|             0|             1|             0|            0|            0|            0|            0|            0|             0|             3|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|            0.75|             0.0|            0.25|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.75|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.25|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             2|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 2|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              1|              0|              1|              0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                  1|                  0|                  1|                  0|                        2|                        0|                        0|                        0|                  0.5|                        1.0|                  0.0|                        0.0|                  0.5|                        0.0|                  0.0|                        0.0|            0|            0|         4727|            0|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            0|         174|         285|         178|         118|         258|          301|         3413|           0|           0|           0|           0|           0|            0|            0|            0.0|            0.0|            1.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           1.0|           1.0|           1.0|           1.0|           1.0|            1.0|            1.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            0|            0|          174|            0|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                0|              21|              29|              30|              25|              29|               31|                9|               0|               0|               0|               0|               0|                0|                0|       0.0|       0.0|     219.0|       0.0|         0.0|         0.0|       218.0|         0.0|      0.0|      0.0|    72.81|      0.0|        0|        0|      301|        0|                 0|                 0|              3393|                 0|                       0|                       0|                      13|                       0|                 0.0|                       0.0|                 0.0|                       0.0|                 1.0|                       1.0|                 0.0|                       0.0|          528|          721|            3|               1|       129720|          417|         8450|         8146|         192|         679|         638|        1656|        4811|        23230|        98514|           1|           0|           1|           4|           8|           50|          353|           1|          12|           7|          38|         179|          741|         7472|          38|          81|          32|         154|         411|         2294|         5136|           0.88|            0.0|           0.06|           0.06|          0.83|          0.88|          0.94|          0.89|          0.89|           0.88|           0.88|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|          0.02|          0.01|          0.02|          0.03|           0.03|           0.07|          0.16|           0.1|          0.05|          0.08|          0.08|           0.09|           0.05|          176|           44|          101|          164|              12|              30|              31|              31|              30|               31|               11|               1|               0|               1|               4|               8|               19|               11|               1|               7|               6|              18|              27|               31|               11|              15|              28|              18|              31|              30|               31|               11|    5201.0|     10.67|     163.0|    501.67|      1167.5|         2.5|        25.0|       117.5|  8990.26|    19.49|   290.98|   889.28|    23230|       50|      741|     2294|             52948|               105|              7241|               938|                   37551|                     209|                     158|                    3195|           61232|                 41113|                0.86|                      0.91|                 0.0|                      0.01|                0.12|                       0.0|                0.02|                      0.08|          667|          778|            3|               1|
# |  1023|    6109| 100001|       1963|    0|     1|  0|1070887|           87|            0|            3|            0|           0|           0|           0|           1|           1|            1|           84|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            3|           0|           0|           0|           0|           0|            0|            0|           0.97|            0.0|           0.03|            0.0|           0.0|           0.0|           0.0|           1.0|           1.0|            1.0|           0.97|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.03|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            4|            0|            1|            0|               0|               0|               0|               1|               1|                1|                1|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                0|            74|             0|             3|             0|           20|            0|            3|            0|           35|            0|            3|            0|           40|            0|            3|            0|       0.5|       0.0|       0.0|       0.0|         0.5|         0.0|         0.0|         0.0|     0.55|      0.0|      0.0|      0.0|        1|        0|        0|        0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.85|                10|              1.97|               1.0|                0.0|                 0|               0.0|               0.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                84|                 0|                 3|                 0|                       0|                       0|                       0|                       0|                0.97|                       0.0|                 0.0|                       0.0|                0.03|                       0.0|                 0.0|                       0.0|        57321|           54|         1497|         1553|        2393|        5929|        1471|        1858|        2206|         5332|        38132|           0|           3|           1|           1|           1|            5|           43|          89|          81|          10|          16|          18|           13|         1270|         126|         204|          59|          51|          64|          309|          740|           0.95|            0.0|           0.02|           0.03|          0.92|          0.95|          0.95|          0.96|          0.96|           0.94|           0.95|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.03|          0.01|          0.01|          0.01|          0.01|            0.0|           0.03|          0.05|          0.03|          0.04|          0.03|          0.03|           0.05|           0.02|          176|           22|           77|          153|              12|              30|              31|              31|              30|               31|               11|               0|               3|               1|               1|               1|                5|               11|              18|              18|               8|               8|              13|               11|                1|              20|              28|              19|              21|              25|               29|               11|           500|            41|           190|           324|           28|            7|           17|           24|            1|            0|            1|            1|            1|            1|            1|            1|   3198.17|      1.83|     37.83|     135.5|      2299.5|         1.0|        17.0|        95.0|  1919.51|     1.83|    36.72|   102.95|     5929|        5|       89|      309|              1.54|               35|             1.42|              1.0|              1.11|                2|             0.32|              1.0|              1.02|                3|             0.13|              1.0|              1.27|               10|             0.75|              1.0|               3.75|               119|              6.05|               2.0|               1.23|                 4|              0.64|               1.0|               1.21|                 7|              0.57|               1.0|               1.81|                22|              1.75|               1.0|             20983|                 8|              1270|               102|                   13743|                      25|                       0|                     494|                0.94|                      0.96|                 0.0|                       0.0|                0.06|                       0.0|                 0.0|                      0.03|          205|          224|            3|               1|             6|             0|             1|             0|            0|            0|            0|            0|            0|             0|             6|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|            0.86|             0.0|            0.14|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.86|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|            0.14|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             1|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              4|              0|              1|              0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                  6|                  0|                  1|                  0|                        0|                        0|                        0|                        0|                 0.86|                        0.0|                  0.0|                        0.0|                 0.14|                        0.0|                  0.0|                        0.0|        79322|            0|         2766|         2111|        3066|        7293|        2802|        2690|        3213|         6627|        53631|           0|           0|           0|           0|           0|            0|            0|         138|         123|          58|          61|          47|           44|         2295|         158|         281|         102|          81|          98|          373|         1018|           0.94|            0.0|           0.03|           0.03|          0.91|          0.95|          0.95|          0.95|          0.96|           0.94|           0.94|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.04|          0.02|          0.02|          0.02|          0.01|           0.01|           0.04|          0.05|          0.04|          0.03|          0.03|          0.03|           0.05|           0.02|          176|            0|          141|          170|              12|              30|              31|              31|              30|               31|               11|               0|               0|               0|               0|               0|                0|                0|              19|              24|              21|              21|              24|               25|                7|              20|              29|              28|              25|              27|               30|               11|   4281.83|       0.0|      78.5|    182.17|      3139.5|         0.0|        59.5|       130.0|  2093.39|      0.0|    41.06|   118.76|     7293|        0|      138|      373|             31653|                 0|              2280|               173|                   18219|                       0|                      14|                     676|                0.93|                      0.96|                 0.0|                       0.0|                0.07|                       0.0|                0.01|                      0.04|          445|          503|            3|               1|       539378|         1127|        38850|        21031|       23760|       61042|       38500|       37515|       48578|        69839|       260144|          14|          28|          25|          28|          58|          130|          844|        3086|        4175|        3074|        2334|        3151|         3925|        19105|        1739|        2319|        1697|        1700|        2212|         3799|         7565|            0.9|            0.0|           0.06|           0.04|          0.83|           0.9|          0.89|           0.9|           0.9|            0.9|            0.9|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.11|          0.06|          0.07|          0.06|          0.06|           0.05|           0.07|          0.06|          0.03|          0.04|          0.04|          0.04|           0.05|           0.03|          177|          127|          185|          185|              12|              30|              31|              31|              30|               31|               12|              10|              15|              19|              16|              26|               30|               11|              21|              30|              31|              31|              30|               31|               11|              21|              30|              31|              31|              30|               31|               11|   46539.0|     47.17|   3290.83|   2244.33|     43539.0|        28.0|      3118.5|      1975.5| 16866.09|    43.14|   664.53|    809.3|    69839|      130|     4175|     3799|            142631|               241|             17531|              1551|                   98827|                     519|                    1138|                    4951|          161954|                105435|                0.88|                      0.94|                 0.0|                       0.0|                0.11|                      0.01|                0.01|                      0.05|         6916|        10975|            3|               0|
# |  1421|    2240| 100001|       2273|    0|     1|  0| 191499|           87|            0|            3|            0|           0|           0|           0|           1|           1|            1|           84|           0|           0|           0|           0|           0|            0|            0|           0|           0|           0|           0|           0|            0|            3|           0|           0|           0|           0|           0|            0|            0|           0.97|            0.0|           0.03|            0.0|           0.0|           0.0|           0.0|           1.0|           1.0|            1.0|           0.97|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|           0.03|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|            4|            0|            1|            0|               0|               0|               0|               1|               1|                1|                1|               0|               0|               0|               0|               0|                0|                0|               0|               0|               0|               0|               0|                0|                1|               0|               0|               0|               0|               0|                0|                0|            74|             0|             3|             0|           20|            0|            3|            0|           35|            0|            3|            0|           40|            0|            3|            0|       0.5|       0.0|       0.0|       0.0|         0.5|         0.0|         0.0|         0.0|     0.55|      0.0|      0.0|      0.0|        1|        0|        0|        0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.0|                1|              0.0|              1.0|               0.0|                0|              0.0|              0.0|               1.85|                10|              1.97|               1.0|                0.0|                 0|               0.0|               0.0|                1.0|                 1|               0.0|               1.0|                0.0|                 0|               0.0|               0.0|                84|                 0|                 3|                 0|                       0|                       0|                       0|                       0|                0.97|                       0.0|                 0.0|                       0.0|                0.03|                       0.0|                 0.0|                       0.0|        37507|           66|         4445|         2947|         705|        2203|        1754|        1800|        3359|         3243|        24443|           1|           1|           1|           2|           3|            5|           53|         322|         159|         162|         141|         268|          179|         3214|         154|         120|         159|         138|         317|          264|         1795|           0.83|            0.0|            0.1|           0.07|           0.6|          0.89|          0.84|          0.86|          0.85|           0.88|           0.83|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.27|          0.06|          0.08|          0.07|          0.07|           0.05|           0.11|          0.13|          0.05|          0.08|          0.07|          0.08|           0.07|           0.06|          176|           21|          181|          174|              12|              30|              31|              31|              30|               31|               11|               1|               1|               1|               2|               2|                5|                9|              20|              30|              29|              30|              30|               31|               11|              20|              28|              27|              28|              29|               31|               11|            95|            10|            46|            44|           14|            1|            8|            5|            1|            1|            1|            1|            1|            1|            1|            1|   2177.33|      2.17|    205.17|     192.0|      2001.5|         1.5|       170.5|       156.5|   1002.2|      1.6|    72.66|     79.3|     3359|        5|      322|      317|              1.49|               20|              1.1|              1.0|              1.03|                2|             0.18|              1.0|              1.01|                6|             0.15|              1.0|              1.18|                6|             0.51|              1.0|                1.8|                21|              1.47|               1.0|               1.03|                 2|              0.18|               1.0|               1.02|                 3|              0.14|               1.0|               1.19|                 6|              0.54|               1.0|             14628|                23|              3194|               268|                    8640|                      26|                      16|                    1297|                0.81|                      0.87|                 0.0|                       0.0|                0.18|                       0.0|                0.01|                      0.13|          238|          298|            3|               0|             1|             0|             1|             0|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|            0|            0|            0|            0|            0|             0|             1|            0|            0|            0|            0|            0|             0|             0|             0.5|             0.0|             0.5|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.5|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.5|            0.0|            0.0|            0.0|            0.0|            0.0|             0.0|             0.0|             1|             0|             1|             0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|                0|                0|                0|                0|                0|                 0|                 1|                0|                0|                0|                0|                0|                 0|                 0|              1|              0|              1|              0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|             1|             0|        0.0|        0.0|        0.0|        0.0|          0.0|          0.0|          0.0|          0.0|       0.0|       0.0|       0.0|       0.0|         0|         0|         0|         0|                  1|                  0|                  1|                  0|                        0|                        0|                        0|                        0|                  0.5|                        0.0|                  0.0|                        0.0|                  0.5|                        0.0|                  0.0|                        0.0|        40459|            1|         4686|         3205|         728|        2281|        1817|        1874|        3444|         3574|        26741|           0|           0|           0|           0|           0|            0|            1|         322|         163|         165|         146|         276|          193|         3421|         154|         123|         163|         144|         327|          327|         1967|           0.84|            0.0|            0.1|           0.07|           0.6|          0.89|          0.85|          0.87|          0.85|           0.87|           0.83|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.27|          0.06|          0.08|          0.07|          0.07|           0.05|           0.11|          0.13|          0.05|          0.08|          0.07|          0.08|           0.08|           0.06|          176|            1|          181|          174|              12|              30|              31|              31|              30|               31|               11|               0|               0|               0|               0|               0|                0|                1|              20|              30|              29|              30|              30|               31|               11|              20|              28|              27|              28|              29|               31|               11|   2286.33|       0.0|    210.83|    206.33|      2077.5|         0.0|       179.0|       158.5|  1078.61|      0.0|    71.44|    94.41|     3574|        0|      322|      327|             15796|                 1|              3391|               297|                    9614|                       0|                      21|                    1419|                0.81|                      0.87|                 0.0|                       0.0|                0.17|                       0.0|                0.02|                      0.13|          234|          300|            3|               0|        58328|          116|         6696|         4828|        2352|        3661|        3645|        3621|        6114|         6996|        31939|           2|           1|           3|           3|           4|           13|           90|         587|         294|         387|         374|         619|          523|         3912|         316|         248|         339|         324|         652|          719|         2230|           0.83|            0.0|            0.1|           0.07|          0.72|          0.87|          0.83|          0.84|          0.83|           0.85|           0.84|           0.0|           0.0|           0.0|           0.0|           0.0|            0.0|            0.0|          0.18|          0.07|          0.09|          0.09|          0.08|           0.06|            0.1|           0.1|          0.06|          0.08|          0.07|          0.09|           0.09|           0.06|          176|           31|          185|          185|              12|              30|              31|              31|              30|               31|               11|               2|               1|               3|               3|               3|               10|                9|              21|              30|              31|              31|              30|               31|               11|              21|              30|              31|              31|              30|               31|               11|   4398.17|      4.33|     464.0|     433.0|      3653.0|         3.0|       455.0|       331.5|  1766.03|     4.37|   130.82|   199.19|     6996|       13|      619|      719|             18165|                35|              3783|               346|                   11962|                      48|                     103|                    1587|           22329|                 13700|                0.81|                      0.87|                 0.0|                       0.0|                0.17|                      0.01|                0.02|                      0.12|          378|          575|            3|               0|
# +------+--------+-------+-----------+-----+------+---+-------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+--------------+--------------+--------------+--------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+----------+----------+----------+----------+----------+----------+----------+----------+-------------------+-------------------+-------------------+-------------------+-------------------------+-------------------------+-------------------------+-------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+-------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+-------------+-------------+---------------+---------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+----------+----------+----------+----------+------------+------------+------------+------------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+----------------+----------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+-------------+-------------+-------------+----------------+


In [ ]:
# test_feature_6 = test_feature_5 \
#                 .join(itm_feature, "item_id", "left") \
#                 .orderBy("user_id", "merchant_id")

In [ ]:
test_feature_6 = test_feature_5 \
                .join(mb_feature, ["merchant_id", "brand_id"], "left") \
                .orderBy("user_id", "merchant_id")

In [ ]:
test_feature_7 = test_feature_6 \
                .join(mc_feature, ["merchant_id", "cat_id"], "left") \
                .orderBy("user_id", "merchant_id")

In [ ]:
# test_feature_8.show()

## Save Parquet

In [ ]:
test_feature_7.coalesce(50) \
               .write.format("parquet") \
               .mode("overwrite") \
               .save("./drive/MyDrive/Colab Notebooks/data/feature_u")